In [2]:
import pandas as pd
import numpy as np
import scipy.stats
import datetime

from scipy import stats
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate,  KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report , matthews_corrcoef
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, ConfusionMatrixDisplay, RocCurveDisplay
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

import itertools
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import utils
from keras.metrics import AUC
import matplotlib.pyplot as plt
from keras import regularizers
import tensorflow as tf


%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'png'

In [3]:
## Open dataset
data = pd.read_excel("DataSet.xlsx")
print(data.shape)
## How many POAF and POAF free
end_point = 'Atrial fibrillation'
print(data[end_point].value_counts(dropna = False))
print(data[end_point].value_counts(dropna = False, normalize=True))

(1305, 55)
Atrial fibrillation
0    1025
1     280
Name: count, dtype: int64
Atrial fibrillation
0    0.785441
1    0.214559
Name: proportion, dtype: float64


### Let's calculate some features

In [47]:
######### Сreatinine clearance
name = 'CC'
data[name] = np.nan
data.loc[:,(name)]= (140-data.loc[:,('Age, years')]) * (data.loc[:,('Height, cm')]/(72 * data.loc[:,('Creatinine, µmol/l')] /88.4))
data.loc[(data["Female"] == 1),(name)]= data.loc[(data["Female"] == 1),(name)] * 0.85


count    893.000000
mean     166.547678
std       43.873711
min       51.673995
25%      137.164846
50%      162.782870
75%      190.906383
max      364.240741
Name: CC, dtype: float64


## Analysis of features (Table 1)

### Analysis of dichotomous features in comparison groups (Table 1)


In [ ]:
features=['Female','CHF III-IV FC','Arterial hypertension','Aortic regurgitation','Mitral regurgitation','Tricuspid regurgitation','Extracardiac arteriopathy','Chronic kidney disease','Aortic stenosis','Сhronic obstructive pulmonary disease',
            'Diabetes mellitus', 'Previous stroke']

for name in features:
    print(f"\n{'='*60}")
    print(f"Feature analysis: {name}")

    # Descriptive statistics
    print("Distribution of values:")

    print(data[name].value_counts(dropna=False))

    print("\nGrouping by end point  (absolute frequencies):")
    print(data.groupby([end_point])[name].value_counts(dropna=False, normalize=False))

    print("\nGrouping by end point (relative frequencies):")
    print(data.groupby([end_point])[name].value_counts(dropna=False, normalize=True))

    # Creating a contingency table
    ct = pd.crosstab(data[end_point], data[name])

    # Fisher's exact test
    oddsratio, pvalue_fisher = stats.fisher_exact(ct)
    print(f'\nFishers exact test - Odds ratio: {oddsratio:.4f}, p-value: {pvalue_fisher:.4f}')

    # Odds ratio and 95% CI
    try:
        table = sm.stats.Table2x2(ct, shift_zeros=False)
        odds_ratio = table.oddsratio
        confint = table.oddsratio_confint()
        print(f'Odds ratio: {odds_ratio:.4f}, 95% CI: [{confint[0]:.4f}, {confint[1]:.4f}]')
    except Exception as e:
        print(f'Error in calculating odds ratio: {e}')

    # Chi-square test
    try:
        chi2, p_chi2, dof, expected = scipy.stats.chi2_contingency(pd.crosstab(data[end_point], data[name]))
        print(f"Chi-square test: p-value = {p_chi2:.4f}")
    except Exception as e:
        print(f'Error in calculating Chi-square test: {e}')

    # Добавляем небольшую паузу для удобства чтения
    print(f"\n{'='*60}")

### Analysis of continuous features in comparison groups (Table 1)

In [ ]:
features = ['Age, years', 'Weight, kg', 'Height, cm', 'BMI, kg/m^2', 'EF LV, %', 'RLVMI, c.u.', 'RVAWRTI, c. u.', 'LV ESD, cm', 'LV EDD, cm', 'Systolic pressure gradient Ao/LV, mm Hg', 'MPAP, mm Hg',
           'LAL, cm', 'LAD, cm', 'Indexed LA volume, ml/m2', 'RAL, cm', 'RAD, cm', 'P, ms', 'PQ, ms\n', 'QRS, ms\n', 'QT, ms\n', 'Creatinine, µmol/l', 'CC',
            "Hemoglobin, g/l","Red blood cells,10^12/l","Leukocytes,10^9/l","Lymphocytes,10^9/l","Total cholesterol,mmol/l","Glucose, mmol/l","Total protein, g/l","Total bilirubin, µmol/l","Triglycerides, mmol/l","Urea, mmol/l","DBP, mm Hg","PTI, %","INR","Platelets,10^9/l","SBP, mm Hg","DBP, mm Hg","Heart rate, beats/min"
]


for name in features:

  print(f"\n{'='*60}")
  print(f"Feature analysis: {name}")

  n1 = data.loc[(data[end_point] == 0), (name)].astype(float).dropna()
  n2 = data.loc[(data[end_point] == 1), (name)].astype(float).dropna()

  print('Mann-Whitney =  %.26f' %  scipy.stats.mannwhitneyu(n1, n2).pvalue)
  print(data[name].dropna().describe())
  print(data.groupby([end_point])[name].describe())

  sns.boxplot(x=end_point, y=name, data=data)
  plt.xlabel("POAF")
  plt.ylabel( name )
  plt.show()

  print(f"\n{'='*60}")







##Development of models based on continuous variables (Table 2)



In [ ]:
#Data preparation
features =[ 'Age, years',  'RAD, cm' , 'Tricuspid regurgitation' ,  'QRS, ms\n' , 'QT, ms\n', 'RR, ms\n' ,'PQ, ms\n', 'P, ms' ,  'LV ESD, cm'
          ]
y_name = "Atrial fibrillation"
rm = 100
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
features.remove(y_name)
n_splits = 10

###Logistic regression (Table 2)


In [ ]:
np.random.seed(rm)
border = 0.195

########### Logistic regression
max_iter = 10000

mean_roc_auc_test = []
mean_sen_test = []
mean_spec_test = []
mean_f1_test = []
mean_ppv_test = []
mean_npv_test = []

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []


for j in range(100):
    rm=j+42
    np.random.seed(rm)
    # We set aside 20% for final testing
    x, x_validate, y, y_validate = train_test_split(      model_data[features],model_data[y_name],
                                                          train_size=0.8,
                                                          stratify=model_data[y_name],
                                                          random_state = rm )
    x = np.array(x)
    y = utils.to_categorical(y)
    x_validate = np.array(x_validate)
    y_validate = utils.to_categorical(y_validate)

    roc_auc_test=[]
    sen_test=[]
    spec_test=[]
    matr_test=[]
    f1_test=[]
    ppv_test=[]
    npv_test=[]
    threshold_test=[]
    skf = StratifiedKFold(n_splits=n_splits )

    # training and cross-validation
    for i, (train_index, test_index) in enumerate(skf.split(x, y[:,1])):

        x_train, x_test = x[train_index,:], x[test_index,:]
        y_train, y_test = y[train_index,:], y[test_index,:]

        model = LogisticRegression( max_iter=max_iter )

        np.random.seed(rm)
        model.fit(x_train, y_train[:,1])
        y_pred_log=model.predict_proba(x_test)
        new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
        new_y = np.array(new_y)
        cMatrix = confusion_matrix(y_test[:,1].astype("int"), new_y)
        sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
        specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
        sen_test.append(sensivity)
        spec_test.append(specifity)
        fpr,tpr,thresholds = roc_curve(y_test[:,1], y_pred_log[:,1] )
        roc_auc = auc(fpr, tpr)
        roc_auc_test.append(roc_auc)
        f1=f1_score(y_test[:,1], new_y)
        f1_test.append(f1)
        ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
        npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

        ppv_test.append(ppv)
        npv_test.append(npv)

    mean_roc_auc_test.append(np.mean(roc_auc_test))
    mean_sen_test.append(np.mean(sen_test))
    mean_spec_test.append(np.mean(spec_test))
    mean_f1_test.append(np.mean(f1_test))
    mean_ppv_test.append(np.mean(ppv_test))
    mean_npv_test.append(np.mean(npv_test))

    model = LogisticRegression( max_iter=max_iter )

    np.random.seed(rm)
    model.fit(x, y[:,1])

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)
    cMatrix = confusion_matrix(y_validate[:,1].astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate[:,1], y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate[:,1], new_y)
    ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Mean area under the ROC curve = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_roc_auc_test),
                                                                       np.mean (mean_roc_auc_test) -
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test)) ,
                                                                       np.mean (mean_roc_auc_test) +
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test) )
                                                                             ))
print('Mean Sensitivity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen_test),
                                                                       np.mean (mean_sen_test) -
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test)) ,
                                                                       np.mean (mean_sen_test) +
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test))
                                                                               ))
print('Mean Specificity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec_test),
                                                                       np.mean (mean_spec_test) - 1.96 * np.std(mean_spec_test) /np.sqrt(len(mean_spec_test)),
                                                                       np.mean (mean_spec_test) + 1.96 * np.std(mean_spec_test)/np.sqrt(len(mean_spec_test) ))
                                                                            )
print('Mean F1-score ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1_test),
                                                                       np.mean (mean_f1_test) - 1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test)) ,
                                                                       np.mean (mean_f1_test) +
                                                                       1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test) ))
                                                                      )

print('Mean PPV ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv_test),
                                                                       np.mean (mean_ppv_test) -
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test)) ,
                                                                       np.mean (mean_ppv_test) +
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test) ))
                                                                      )

print('Mean NPV  ={:.4f} 95% [ {:.4f}, {:.4f}]'.format(np.mean (mean_npv_test),
                                                                   np.mean (mean_npv_test) -
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)),
                                                                   np.mean (mean_npv_test) +
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)))
                                                                  )

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )

###XGBoost (Table 2)


In [ ]:
np.random.seed(rm)
border = 0.41

########### XGBoost
lr=0.03
m_d=2
n_e=200
spw=3
child_weight=1
sub=0.7
max_step=1
gam=2
method ='exact'
boost= 'gbtree'

mean_roc_auc_test = []
mean_sen_test = []
mean_spec_test = []
mean_f1_test = []
mean_ppv_test = []
mean_npv_test = []

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []


for j in range(100):
    rm=j+42
    np.random.seed(rm)
    # We set aside 20% for final testing
    x, x_validate, y, y_validate = train_test_split(      model_data[features],model_data[y_name],
                                                          train_size=0.8,
                                                          stratify=model_data[y_name],
                                                          random_state = rm )
    x = np.array(x)
    y = utils.to_categorical(y)
    x_validate = np.array(x_validate)
    y_validate = utils.to_categorical(y_validate)

    roc_auc_test=[]
    sen_test=[]
    spec_test=[]
    matr_test=[]
    f1_test=[]
    ppv_test=[]
    npv_test=[]
    threshold_test=[]
    skf = StratifiedKFold(n_splits=n_splits )

    # training and cross-validation
    for i, (train_index, test_index) in enumerate(skf.split(x, y[:,1])):

        x_train, x_test = x[train_index,:], x[test_index,:]
        y_train, y_test = y[train_index,:], y[test_index,:]

        model = xgb.XGBClassifier(learning_rate=lr,  eval_metric = "auc",
                                      scale_pos_weight = spw,    max_depth=m_d,  n_estimators=n_e,  random_state=rm,
                                      verbosity=0 , objective= 'binary:logistic', booster= boost,
                                      tree_method =method, max_delta_step=max_step, gamma=gam,
                                      min_child_weight=child_weight,
                                      subsample=sub, colsample_bylevel=1,
                                      n_jobs=-1, use_label_encoder=False
                                     )

        np.random.seed(rm)
        model.fit(x_train, y_train[:,1])
        y_pred_log=model.predict_proba(x_test)
        new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
        new_y = np.array(new_y)
        cMatrix = confusion_matrix(y_test[:,1].astype("int"), new_y)
        sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
        specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
        sen_test.append(sensivity)
        spec_test.append(specifity)
        fpr,tpr,thresholds = roc_curve(y_test[:,1], y_pred_log[:,1] )
        roc_auc = auc(fpr, tpr)
        roc_auc_test.append(roc_auc)
        f1=f1_score(y_test[:,1], new_y)
        f1_test.append(f1)
        ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
        npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

        ppv_test.append(ppv)
        npv_test.append(npv)

    mean_roc_auc_test.append(np.mean(roc_auc_test))
    mean_sen_test.append(np.mean(sen_test))
    mean_spec_test.append(np.mean(spec_test))
    mean_f1_test.append(np.mean(f1_test))
    mean_ppv_test.append(np.mean(ppv_test))
    mean_npv_test.append(np.mean(npv_test))

    model = xgb.XGBClassifier(learning_rate=lr,  eval_metric = "auc",
                                      scale_pos_weight = spw,    max_depth=m_d,  n_estimators=n_e,  random_state=rm,
                                      verbosity=0 , objective= 'binary:logistic', booster= boost,
                                      tree_method =method, max_delta_step=max_step, gamma=gam,
                                      min_child_weight=child_weight,
                                      subsample=sub, colsample_bylevel=1,
                                      n_jobs=-1, use_label_encoder=False
                                     )

    np.random.seed(rm)
    model.fit(x, y[:,1])

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)
    cMatrix = confusion_matrix(y_validate[:,1].astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate[:,1], y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate[:,1], new_y)
    ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Mean area under the ROC curve = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_roc_auc_test),
                                                                       np.mean (mean_roc_auc_test) -
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test)) ,
                                                                       np.mean (mean_roc_auc_test) +
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test) )
                                                                             ))
print('Mean Sensitivity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen_test),
                                                                       np.mean (mean_sen_test) -
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test)) ,
                                                                       np.mean (mean_sen_test) +
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test))
                                                                               ))
print('Mean Specificity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec_test),
                                                                       np.mean (mean_spec_test) - 1.96 * np.std(mean_spec_test) /np.sqrt(len(mean_spec_test)),
                                                                       np.mean (mean_spec_test) + 1.96 * np.std(mean_spec_test)/np.sqrt(len(mean_spec_test) ))
                                                                            )
print('Mean F1-score ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1_test),
                                                                       np.mean (mean_f1_test) - 1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test)) ,
                                                                       np.mean (mean_f1_test) +
                                                                       1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test) ))
                                                                      )

print('Mean PPV ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv_test),
                                                                       np.mean (mean_ppv_test) -
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test)) ,
                                                                       np.mean (mean_ppv_test) +
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test) ))
                                                                      )

print('Mean NPV  ={:.4f} 95% [ {:.4f}, {:.4f}]'.format(np.mean (mean_npv_test),
                                                                   np.mean (mean_npv_test) -
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)),
                                                                   np.mean (mean_npv_test) +
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)))
                                                                  )

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )

###Random Forest (Table 2)

In [ ]:
np.random.seed(rm)
border = 0.22

### Random Forest
m_d1=8
n_e1=110
min_leaf = 0.01

mean_roc_auc_test = []
mean_sen_test = []
mean_spec_test = []
mean_f1_test = []
mean_ppv_test = []
mean_npv_test = []

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []


for j in range(100):
    rm=j+42
    np.random.seed(rm)
    # We set aside 20% for final testing
    x, x_validate, y, y_validate = train_test_split(      model_data[features],model_data[y_name],
                                                          train_size=0.8,
                                                          stratify=model_data[y_name],
                                                          random_state = rm )
    x = np.array(x)
    y = utils.to_categorical(y)
    x_validate = np.array(x_validate)
    y_validate = utils.to_categorical(y_validate)

    roc_auc_test=[]
    sen_test=[]
    spec_test=[]
    matr_test=[]
    f1_test=[]
    ppv_test=[]
    npv_test=[]
    threshold_test=[]
    skf = StratifiedKFold(n_splits=n_splits )

    # training and cross-validation
    for i, (train_index, test_index) in enumerate(skf.split(x, y[:,1])):

        x_train, x_test = x[train_index,:], x[test_index,:]
        y_train, y_test = y[train_index,:], y[test_index,:]

        model = RandomForestClassifier(random_state=rm, n_estimators=n_e1, max_depth=m_d1
                                          ,min_samples_leaf = min_leaf
                                          )

        np.random.seed(rm)
        model.fit(x_train, y_train[:,1])
        y_pred_log=model.predict_proba(x_test)
        new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
        new_y = np.array(new_y)
        cMatrix = confusion_matrix(y_test[:,1].astype("int"), new_y)
        sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
        specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
        sen_test.append(sensivity)
        spec_test.append(specifity)
        fpr,tpr,thresholds = roc_curve(y_test[:,1], y_pred_log[:,1] )
        roc_auc = auc(fpr, tpr)
        roc_auc_test.append(roc_auc)
        f1=f1_score(y_test[:,1], new_y)
        f1_test.append(f1)
        ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
        npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

        ppv_test.append(ppv)
        npv_test.append(npv)

    mean_roc_auc_test.append(np.mean(roc_auc_test))
    mean_sen_test.append(np.mean(sen_test))
    mean_spec_test.append(np.mean(spec_test))
    mean_f1_test.append(np.mean(f1_test))
    mean_ppv_test.append(np.mean(ppv_test))
    mean_npv_test.append(np.mean(npv_test))

    model = RandomForestClassifier(random_state=rm, n_estimators=n_e1, max_depth=m_d1
                                          ,min_samples_leaf = min_leaf
                                          )

    np.random.seed(rm)
    model.fit(x, y[:,1])

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)
    cMatrix = confusion_matrix(y_validate[:,1].astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate[:,1], y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate[:,1], new_y)
    ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Mean area under the ROC curve = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_roc_auc_test),
                                                                       np.mean (mean_roc_auc_test) -
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test)) ,
                                                                       np.mean (mean_roc_auc_test) +
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test) )
                                                                             ))
print('Mean Sensitivity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen_test),
                                                                       np.mean (mean_sen_test) -
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test)) ,
                                                                       np.mean (mean_sen_test) +
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test))
                                                                               ))
print('Mean Specificity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec_test),
                                                                       np.mean (mean_spec_test) - 1.96 * np.std(mean_spec_test) /np.sqrt(len(mean_spec_test)),
                                                                       np.mean (mean_spec_test) + 1.96 * np.std(mean_spec_test)/np.sqrt(len(mean_spec_test) ))
                                                                            )
print('Mean F1-score ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1_test),
                                                                       np.mean (mean_f1_test) - 1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test)) ,
                                                                       np.mean (mean_f1_test) +
                                                                       1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test) ))
                                                                      )

print('Mean PPV ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv_test),
                                                                       np.mean (mean_ppv_test) -
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test)) ,
                                                                       np.mean (mean_ppv_test) +
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test) ))
                                                                      )

print('Mean NPV  ={:.4f} 95% [ {:.4f}, {:.4f}]'.format(np.mean (mean_npv_test),
                                                                   np.mean (mean_npv_test) -
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)),
                                                                   np.mean (mean_npv_test) +
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)))
                                                                  )

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )

##Dichotomization of continuous predictors (Table 3)


### Search for the cutoff threshold - Min(p-value) (Table 3)

In [ ]:
features =[ {"name":'Age, years', "sign":">" },
            {"name":'LV ESD, cm', "sign":">" },
            {"name":'RAD, cm', "sign":"<" },
            {"name":'QRS, ms\n', "sign":">" },
            {"name":'QT, ms\n', "sign":">" },
            {"name":'RR, ms\n', "sign":">" },
            {"name":'PQ, ms\n', "sign":">" },
            {"name":'P, ms', "sign":">" },
          ]
y_name = "Atrial fibrillation"

name1='isParameter'
for feature in features:
  name = feature["name"]
  sign = feature["sign"]
  print( "Feature: " + name )

  pmin = 1
  #### min (p-value) & max (ОШ)
  d=data.loc[:,(name, y_name)].dropna()
  d[name1]=None

  x0=d[name].min()
  x1=d[name].max()

  delta = (x1-x0)/100

  if sign == '>':
      xcurrent=x0+delta
      xp=x0
  else:
      xcurrent=x1-delta
      xp=x1

  yPV=[]
  xPV=[]
  ##########  Minimizing  P-value
  while (xcurrent < x1) & (xcurrent > x0):
      d[name1]=None
      if sign == '>':
          d.loc[(d[name] > xcurrent ),(name1)]= 1
          d.loc[(d[name] <= xcurrent ),(name1)]= 0
      else:
          d.loc[(d[name] < xcurrent ),(name1)]= 1
          d.loc[(d[name] >= xcurrent ),(name1)]= 0

      ct=pd.crosstab(d[y_name], d[(name1)])
      ### Chi-square test for p-value
      pvalue=scipy.stats.chi2_contingency(ct)[1]
      if (pvalue < pmin):
          pmin=pvalue
          xp=xcurrent

      if sign == '>':
          xcurrent += delta
      else:
          xcurrent -= delta

      xPV.append(xcurrent)
      yPV.append(pvalue)

  ##########   P-value
  print(f'Cutoff threshold = {xp}  Min(p-value)= {pmin} ' )

  ## Calculating additional metrics for the threshold with the minimum AUC
  if sign == '>':
      d.loc[(d[name] > xp ),(name1)]= 1
      d.loc[(d[name] <= xp ),(name1)]= 0
  else:
      d.loc[(d[name] < xp ),(name1)]= 1
      d.loc[(d[name] >= xp ),(name1)]= 0

  table = sm.stats.Table2x2(pd.crosstab(d[y_name], d[(name1)]), shift_zeros=False)
  print(f'OdssRation= {table.oddsratio}, 95%CI = {table.oddsratio_confint()} \np-value OR= {table.oddsratio_pvalue()} ')


  ################## Сalculating AUC
  if sign == '>':
      data.loc[(data[name] > xp ),(name1)]= 1
      data.loc[(data[name] <= xp ),(name1)]= 0
  else:
      data.loc[(data[name] < xp ),(name1)]= 1
      data.loc[(data[name] >= xp ),(name1)]= 0

  columns = [(name1)]
  columns.append(y_name)
  model_data = data[columns]
  model_data = model_data.dropna()
  columns.remove(y_name)

  x = np.array(model_data[columns])  #Select only significant columns
  y1 = np.array(model_data[y_name].astype('int')) ##Column of class labels
  y = utils.to_categorical(y1)

  n_splits=10
  kf = StratifiedKFold(n_splits=n_splits )
  roc_auc_max_log=0
  roc_auc_test=[]
  for train_index, test_index in kf.split(x,y[:,1]):

      x_train, x_test = x[train_index,:], x[test_index,:]
      y_train, y_test = y[train_index,:], y[test_index,:]

      ########### Logistic regression
      model = LogisticRegression(solver = 'lbfgs',  max_iter = 10000, C = 1,  penalty='l2')
      model.fit(x_train, y_train[:,1])
      y_pred_log=model.predict_proba(x_test)
      fpr,tpr,_ = roc_curve(y_test[:,1], y_pred_log[:,1] )
      roc_auc = auc(fpr, tpr)
      roc_auc_test.append(roc_auc)

  print('AUC (p-value) = %.4f' % np.mean (roc_auc_test) )
  model = LogisticRegression(solver = 'lbfgs',  max_iter = 10000, C = 1,  penalty='l2')
  model.fit(x, y[:,1])
  print('Coef(p-value)', model.coef_[0])
  print('='*60 + "\n")


### Search for the cutoff threshold - Max(AUC) (Table 3)


In [ ]:
features =[ {"name":'Age, years', "sign":">" },
            {"name":'LV ESD, cm', "sign":">" },
            {"name":'RAD, cm', "sign":"<" },
            {"name":'QRS, ms\n', "sign":">" },
            {"name":'QT, ms\n', "sign":">" },
            {"name":'RR, ms\n', "sign":">" },
            {"name":'PQ, ms\n', "sign":">" },
            {"name":'P, ms', "sign":">" },
          ]
y_name = "Atrial fibrillation"

name1='isParameter'
for feature in features:
  name = feature["name"]
  sign = feature["sign"]
  print( "Feature: " + name )

  d=data.loc[:,(name,y_name)].dropna()
  d[name1]=None

  x0=d[name].min()
  x1=d[name].max()
  delta = (x1-x0)/100

  if sign == '>':
      xcurrent=x0+delta
      xauc=x0
  else:
      xcurrent=x1-delta
      xauc=x1

  aucmax=0
  pvaluemin=1

  xAUC=[]
  yAUC=[]
  ##########  Мax(AUC)
  while (xcurrent < x1) & (xcurrent > x0):
      data.loc[:,(name1)]=None

      if sign == '>':
          data.loc[(data[name] > xcurrent ),(name1)]= 1
          data.loc[(data[name] <= xcurrent ),(name1)]= 0
      else:
          data.loc[(data[name] < xcurrent ),(name1)]= 1
          data.loc[(data[name] >= xcurrent ),(name1)]= 0

      columns = [(name1)]
      columns.append(y_name)
      model_data = data[columns]
      model_data = model_data.dropna()
      columns.remove(y_name)

      x = np.array(model_data[columns])
      y1 = np.array(model_data[y_name].astype('int'))
      y = utils.to_categorical(y1)

      n_splits=7
      kf = StratifiedKFold(n_splits=n_splits )
      roc_auc_max_log=0
      roc_auc_test=[]
      for train_index, test_index in kf.split(x,y[:,1]):

          x_train, x_test = x[train_index,:], x[test_index,:]
          y_train, y_test = y[train_index,:], y[test_index,:]

          ########### Logistic regression
          model = LogisticRegression(solver = 'lbfgs',  max_iter = 10000, C = 300
                                    )
          model.fit(x_train, y_train[:,1])
          y_pred_log=model.predict_proba(x_test)
          fpr,tpr,_ = roc_curve(y_test[:,1], y_pred_log[:,1] )
          roc_auc = auc(fpr, tpr)
          roc_auc_test.append(roc_auc)
      if ( (aucmax < np.mean (roc_auc_test)) ):
          aucmax=np.mean (roc_auc_test)
          xauc=xcurrent

      if sign == '>':
          xcurrent += delta
      else:
          xcurrent -= delta

      xAUC.append(xcurrent)
      yAUC.append(np.mean (roc_auc_test))
          ##########   MAX(AUC)
  print(f'Cutoff threshold = {xauc}  Max(AUC)= {aucmax} Min={x0} Max={x1} ')

  ##Calculate additional metrics for the threshold with the maximum AUC
  if sign == '>':
          data.loc[(data[name] > xauc ),(name1)]= 1
          data.loc[(data[name] <= xauc ),(name1)]= 0
  else:
          data.loc[(data[name] < xauc ),(name1)]= 1
          data.loc[(data[name] >= xauc ),(name1)]= 0

  ###Chi-square test for p-value
  pvalue=scipy.stats.chi2_contingency(pd.crosstab(data[y_name], data[(name1)]))[1]
  print(f'Pvalue chi-square test = {pvalue}')
  ### Odds ratio

  table = sm.stats.Table2x2(pd.crosstab(data[y_name], data[(name1)]), shift_zeros=False)
  print(f'OdssRation= {table.oddsratio}, 95%CI = {table.oddsratio_confint()} \np-value OR= {table.oddsratio_pvalue()} ')

  print('='*60 + "\n")


### Search for the cutoff threshold - Centroid (Table 3)


In [ ]:
features =[ {"name":'Age, years', "sign":">" },
            {"name":'LV ESD, cm', "sign":">" },
            {"name":'RAD, cm', "sign":"<" },
            {"name":'QRS, ms\n', "sign":">" },
            {"name":'QT, ms\n', "sign":">" },
            {"name":'RR, ms\n', "sign":">" },
            {"name":'PQ, ms\n', "sign":">" },
            {"name":'P, ms', "sign":">" },
          ]
y_name = "Atrial fibrillation"

name1='isParameter'
for feature in features:
  name = feature["name"]
  sign = feature["sign"]
  print( "Feature: " + name )

  d=data.loc[:,(name,y_name)].dropna()
  d[name1]=None

  a = d.groupby([y_name])[(name)].describe()
  mean1 = a.iloc[1]["50%"]
  mean0 = a.iloc[0]["50%"]
  xcenter=(mean1 + mean0)/2
  print('Порог=',xcenter)

  if sign == '>':
          d.loc[(d[name] > xcenter ),(name1)]= 1
          d.loc[(d[name] <= xcenter ),(name1)]= 0
  else:
          d.loc[(d[name] < xcenter ),(name1)]= 1
          d.loc[(d[name] >= xcenter ),(name1)]= 0
  ### Chi-quadrant test
  pvalue=scipy.stats.chi2_contingency(pd.crosstab(d[y_name], d[(name1)]))[1]
  print(f'pvalue хи-квадрат теста = {pvalue}')
  ### Odds ratio

  table = sm.stats.Table2x2(pd.crosstab(d[y_name], d[(name1)]), shift_zeros=False)
  print(f'OdssRation= {table.oddsratio}, 95%CI = {table.oddsratio_confint()} \np-value OR= {table.oddsratio_pvalue()} ')

  ################## Calculating AUC
  if sign == '>':
      data.loc[(data[name] > xcenter ),(name1)]= 1
      data.loc[(data[name] <= xcenter ),(name1)]= 0
  else:
      data.loc[(data[name] < xcenter ),(name1)]= 1
      data.loc[(data[name] >= xcenter ),(name1)]= 0

  columns = [(name1)]
  columns.append(y_name)
  model_data = data[columns]
  model_data = model_data.dropna()
  columns.remove(y_name)

  x = np.array(model_data[columns])
  y1 = np.array(model_data[y_name].astype('int'))
  y = utils.to_categorical(y1)

  n_splits=10
  kf = StratifiedKFold(n_splits=n_splits )
  roc_auc_max_log=0
  roc_auc_test=[]
  for train_index, test_index in kf.split(x,y[:,1]):

      x_train, x_test = x[train_index,:], x[test_index,:]
      y_train, y_test = y[train_index,:], y[test_index,:]

      ########### Logistic regression
      model = LogisticRegression(solver = 'lbfgs',  max_iter = 10000, C = 1,  penalty='l2')
      model.fit(x_train, y_train[:,1])
      y_pred_log=model.predict_proba(x_test)
      fpr,tpr,_ = roc_curve(y_test[:,1], y_pred_log[:,1] )
      roc_auc = auc(fpr, tpr)
      roc_auc_test.append(roc_auc)

  print('AUC = %.4f' % np.mean (roc_auc_test) )

  print('='*60 + "\n")



### Search for the cutoff threshold - SHAP (Table 3)

In [ ]:
model = xgb.XGBClassifier( learning_rate=lr,  eval_metric = "auc",
                              scale_pos_weight = spw,    max_depth=m_d,  n_estimators=n_e,  random_state=rm,
                              verbosity=0 , objective= 'binary:logistic', booster= boost,
                              tree_method =method, max_delta_step=max_step,
                              gamma=gam, min_child_weight=child_weight,
                              subsample=sub, colsample_bylevel=1,
                                 n_jobs=-1 , use_label_encoder=False
                             )

features =[ 'Age, years',  'RAD, cm' , 'LV ESD, cm',
            'QT, ms\n', 'QRS, ms\n', 'PQ, ms\n',  'RR, ms\n' , 'P, ms' ,
            'Tricuspid regurgitation' ,
            ]
y_name = "Atrial fibrillation"
rm = 100

np.random.seed(rm)
features.append(y_name)
model_data = data[features]
model_data = model_data.dropna()
features.remove(y_name)
model_data_copy=model_data.copy()
model_data_copy=model_data_copy[features]

x_all = np.array(model_data[features])
y1 = np.array(model_data[y_name].astype('int'))
y_all = utils.to_categorical(y1)

model.fit(x_all, y_all[:,1])


In [ ]:
import shap
plt.rcParams.update({'font.size': 10})

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(x_all)
shap_values_df = pd.DataFrame(shap_values )


plt.figure(figsize=[11,26],dpi = 100)

#####Age
plt.subplot(9, 2, 1)
pp = pd.DataFrame({'X': np.array(model_data[features[0]])
                   , 'shap':np.array(shap_values_df[0])
                  })
m=np.max(pp['shap'])
xx=min_age_above_0_2 = pp.loc[pp['shap'] > m/3, 'X'].min()
yy=np.min(pp['shap'])
print(xx, yy, m)
plt.subplots_adjust(wspace=0.4, hspace=0.5)
plt.minorticks_on()
plt.grid()
plt.scatter(  model_data[features[0]],shap_values_df[0], s = 2  )
plt.xlim(( np.min(model_data[features[0]])-np.min(model_data[features[0]])/20,
          np.max(model_data[features[0]])+np.max(model_data[features[0]])/20))
plt.xlabel(features[0])


plt.ylabel('SHAP value')
plt.axvline(x=xx, color='blue', linestyle='--')
plt.axhline(y=0, color='black')
plt.axhline(y=m/2, color='black', linestyle='--')
plt.text(xx+2, yy, f'{int(xx):}', color='blue', ha='center', va='bottom', fontsize=10)

#####  RAD
plt.subplot(9, 2, 2)
pp = pd.DataFrame({'X': np.array(model_data[features[1]])
                   , 'shap':np.array(shap_values_df[1])
                  })
m=np.max(pp['shap'])

xx=min_age_above_0_2 = pp.loc[pp['shap'] > m/10, 'X'].min()
yy=np.min(pp['shap'])
print(xx, yy, m)
plt.subplots_adjust(wspace=0.4, hspace=0.5)
plt.minorticks_on()
plt.grid()
plt.scatter(  model_data[features[1]],shap_values_df[1], s = 2  )
plt.xlim([ np.min(model_data[features[1]])-np.min(model_data[features[1]])/20,
          np.max(model_data[features[1]])+np.max(model_data[features[1]])/20])
plt.xlabel(features[1])


plt.ylabel('SHAP value')
plt.axvline(x=xx, color='blue', linestyle='--')
xx1=5.3
plt.axvline(x=xx1, color='red', linestyle='--')

plt.axhline(y=0, color='black')
plt.axhline(y=m/2, color='black', linestyle='--')
plt.text(xx+0.2, yy, f'{xx:.1f}', color='blue', ha='center', va='bottom', fontsize=10)
plt.text(xx1+0.2, yy, f'{xx1:.1f}', color='red', ha='center', va='bottom', fontsize=10)

#####  LV ESD
plt.subplot(9, 2, 3)
pp = pd.DataFrame({'X': np.array(model_data[features[2]])
                   , 'shap':np.array(shap_values_df[2])
                  })
m=np.max(pp['shap'])

xx=min_age_above_0_2 = pp.loc[pp['shap'] > m/3, 'X'].min()
yy=np.min(pp['shap'])
print(xx, yy, m)
plt.subplots_adjust(wspace=0.4, hspace=0.5)
plt.minorticks_on()
plt.grid()
plt.scatter(  model_data[features[2]],shap_values_df[2], s = 2  )
plt.xlim([ np.min(model_data[features[2]])-np.min(model_data[features[2]])/20,
          np.max(model_data[features[2]])+np.max(model_data[features[2]])/20])
plt.xlabel(features[2])

plt.ylabel('SHAP value')
plt.axvline(x=xx, color='blue', linestyle='--')
xx1=4.1
plt.axvline(x=xx1, color='red', linestyle='--')
xx2=5
plt.axvline(x=xx2, color='blue', linestyle='--')
plt.axhline(y=0, color='black')
plt.axhline(y=m/3, color='black', linestyle='--')
plt.text(xx+0.2, yy, f'{xx:.1f}', color='blue', ha='center', va='bottom', fontsize=10)
plt.text(xx1+0.2, yy, f'{xx1:.1f}', color='red', ha='center', va='bottom', fontsize=10)
plt.text(xx1+0.2, yy, f'{xx1:.1f}', color='red', ha='center', va='bottom', fontsize=10)

#####  QT, ms
plt.subplot(9, 2, 4)
pp = pd.DataFrame({'X': np.array(model_data[features[3]])
                   , 'shap':np.array(shap_values_df[3])
                  })
m=np.max(pp['shap'])

xx=390
yy=np.min(pp['shap'])
print(xx, yy, m)
plt.subplots_adjust(wspace=0.4, hspace=0.5)
plt.minorticks_on()
plt.grid()
plt.scatter(  model_data[features[3]],shap_values_df[3], s = 2  )
plt.xlim([ np.min(model_data[features[3]])-np.min(model_data[features[3]])/20,
          np.max(model_data[features[3]])+np.max(model_data[features[3]])/20])
plt.xlabel(features[3])
plt.ylabel('SHAP value')
plt.axvline(x=xx, color='blue', linestyle='--')
plt.axhline(y=0, color='black')
plt.axhline(y=m/3, color='black', linestyle='--')
plt.text(xx+30, yy, f'{xx:.1f}', color='blue', ha='center', va='bottom', fontsize=10)

#### QRS, ms
plt.subplot(9, 2, 5)
pp = pd.DataFrame({'X': np.array(model_data[features[4]])
                   , 'shap':np.array(shap_values_df[4])
                  })
m=np.max(pp['shap'])
xx=min_age_above_0_2 = pp.loc[pp['shap'] > m/3, 'X'].max()
yy=np.min(pp['shap'])
print(xx, yy, m)
plt.subplots_adjust(wspace=0.4, hspace=0.5)
plt.minorticks_on()
plt.grid()
plt.scatter(  model_data[features[4]],shap_values_df[4], s = 2  )
plt.xlim([ np.min(model_data[features[4]])-np.min(model_data[features[4]])/20,
          np.max(model_data[features[4]])+np.max(model_data[features[4]])/20])
plt.xlabel(features[4])
plt.ylabel('SHAP value')
plt.axvline(x=xx, color='blue', linestyle='--')
plt.axhline(y=0, color='black')
plt.axhline(y=m/3, color='black', linestyle='--')
plt.text(xx+5, yy, f'{int(xx):}', color='blue', ha='center', va='bottom', fontsize=10)

#### PQ, ms
plt.subplot(9, 2, 6)
pp = pd.DataFrame({'X': np.array(model_data[features[5]])
                   , 'shap':np.array(shap_values_df[5])
                  })
m=np.max(pp['shap'])
xx=min_age_above_0_2 = pp.loc[pp['shap'] > m/3, 'X'].min()
yy=np.min(pp['shap'])
print(xx, yy, m)
plt.subplots_adjust(wspace=0.4, hspace=0.5)
plt.minorticks_on()
plt.grid()
plt.scatter(  model_data[features[5]],shap_values_df[5], s = 2  )
plt.xlim([ np.min(model_data[features[5]])-np.min(model_data[features[5]])/20,
          np.max(model_data[features[5]])+np.max(model_data[features[5]])/20])
plt.xlabel(features[5])
plt.ylabel('SHAP value')
plt.axvline(x=xx, color='blue', linestyle='--')
xx1=210
plt.axvline(x=xx1, color='red', linestyle='--')
plt.axhline(y=0, color='black')
plt.axhline(y=m/3, color='black', linestyle='--')
plt.text(xx+15, yy, f'{int(xx):}', color='blue', ha='center', va='bottom', fontsize=10)
plt.text(xx1+15, yy, f'{int(xx1):}', color='red', ha='center', va='bottom', fontsize=10)

#### RR, ms
plt.subplot(9, 2, 7)
pp = pd.DataFrame({'X': np.array(model_data[features[6]])
                   , 'shap':np.array(shap_values_df[6])
                  })
m=np.max(pp['shap'])

xx=min_age_above_0_2 = pp.loc[pp['shap'] > m/3, 'X'].min()
yy=np.min(pp['shap'])
print(xx, yy, m)
plt.subplots_adjust(wspace=0.4, hspace=0.5)
plt.minorticks_on()
plt.grid()
plt.scatter(  model_data[features[6]],shap_values_df[6], s = 2  )
plt.xlim([ np.min(model_data[features[6]])-np.min(model_data[features[6]])/20,
          np.max(model_data[features[6]])+np.max(model_data[features[6]])/20])
plt.xlabel(features[6])
plt.ylabel('SHAP value')
plt.axvline(x=xx, color='blue', linestyle='--')
xx1=750
plt.axvline(x=xx1, color='red', linestyle='--')
xx2=880
plt.axvline(x=xx2, color='blue', linestyle='--')
xx3=1000
plt.axvline(x=xx3, color='red', linestyle='--')
xx4=1100
plt.axvline(x=xx4, color='blue', linestyle='--')
plt.axhline(y=0, color='black')
plt.axhline(y=m/3, color='black', linestyle='--')
plt.text(xx-60, yy, f'{int(xx):}', color='blue', ha='center', va='bottom', fontsize=10)
plt.text(xx1+60, yy, f'{int(xx1):}', color='red', ha='center', va='bottom', fontsize=10)
plt.text(xx2+60, yy, f'{int(xx2):}', color='blue', ha='center', va='bottom', fontsize=10)
plt.text(xx3+60, yy, f'{int(xx3):}', color='red', ha='center', va='bottom', fontsize=10)

#### P, ms
plt.subplot(9, 2, 8)
pp = pd.DataFrame({'X': np.array(model_data[features[7]])
                   , 'shap':np.array(shap_values_df[7])
                  })
m=np.max(pp['shap'])
xx=min_age_above_0_2 = pp.loc[pp['shap'] > m/3, 'X'].min()
yy=np.min(pp['shap'])
print(xx, yy, m)
plt.subplots_adjust(wspace=0.4, hspace=0.5)
plt.minorticks_on()
plt.grid()
plt.scatter(  model_data[features[7]],shap_values_df[7], s = 2  )
plt.xlim([ np.min(model_data[features[7]])-np.min(model_data[features[7]])/20,
          np.max(model_data[features[7]])+np.max(model_data[features[7]])/20])
plt.xlabel(features[7])
plt.ylabel('SHAP value')
plt.axvline(x=xx, color='blue', linestyle='--')
plt.axhline(y=0, color='black')
plt.axhline(y=m/3, color='black', linestyle='--')
plt.text(xx+5, yy, f'{int(xx):}', color='blue', ha='center', va='bottom', fontsize=10)


##Development of models based on categorical variables (Table 4)

###Model Minimum p-value (Table 4)

In [ ]:
name='cat Age'
data[name]=None
data.loc[(data['Age, years'] >=60 ),(name)]= 1
data.loc[(data['Age, years'] < 60 ),(name)]= 0

name='cat RAD'
data[name]=None
data.loc[(data['RAD, cm'] >= 4.17 ) , (name)]= 1
data.loc[(data['RAD, cm'] < 4.17 ) , (name)]= 0

name='cat LV ESD'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 3), (name)]= 1
data.loc[(data['LV ESD, cm'] < 3 ),(name)]= 0

name='cat QRS'
data[name]=None
data.loc[(data['QRS, ms\n'] < 81),(name)]= 1
data.loc[(data['QRS, ms\n'] >= 81 ),(name)]= 0

name='cat QT'
data[name]=None
data.loc[(data['QT, ms\n'] >= 420),(name)]= 1
data.loc[(data['QT, ms\n'] < 420 ),(name)]= 0

name='cat PQ'
data[name]=None
data.loc[(data['PQ, ms\n'] >= 163), (name)]= 1
data.loc[(data['PQ, ms\n'] < 163 ), (name)]= 0

name='cat RR'
data[name]=None
data.loc[ (data['RR, ms\n'] >= 882), (name)]= 1
data.loc[ (data['RR, ms\n'] < 882), (name)]= 0

name='cat P'
data[name]=None
data.loc[(data['P, ms'] >= 120),(name)]= 1
data.loc[(data['P, ms'] < 120 ),(name)]= 0


features =[ 'cat Age',  'cat RAD' , 'Tricuspid regurgitation' ,  'cat QRS' , 'cat QT', 'cat RR' ,'cat PQ', 'cat P' ,  'cat LV ESD'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.2
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
features.remove(y_name)
n_splits = 10

np.random.seed(rm)

########### LogisticRegression
max_iter = 100

mean_roc_auc_test = []
mean_sen_test = []
mean_spec_test = []
mean_f1_test = []
mean_ppv_test = []
mean_npv_test = []

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []


for j in range(100):
    rm=j+42
    np.random.seed(rm)
    # We set aside 20% for final testing
    x, x_validate, y, y_validate = train_test_split(      model_data[features],model_data[y_name],
                                                          train_size=0.8,
                                                          stratify=model_data[y_name],
                                                          random_state = rm )
    x = np.array(x)
    y = utils.to_categorical(y)
    x_validate = np.array(x_validate)
    y_validate = utils.to_categorical(y_validate)

    roc_auc_test=[]
    sen_test=[]
    spec_test=[]
    matr_test=[]
    f1_test=[]
    ppv_test=[]
    npv_test=[]
    threshold_test=[]
    skf = StratifiedKFold(n_splits=n_splits )

    # training and cross-validation
    for i, (train_index, test_index) in enumerate(skf.split(x, y[:,1])):

        x_train, x_test = x[train_index,:], x[test_index,:]
        y_train, y_test = y[train_index,:], y[test_index,:]

        model = LogisticRegression( max_iter=max_iter )


        np.random.seed(rm)
        model.fit(x_train, y_train[:,1])
        y_pred_log=model.predict_proba(x_test)
        new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
        new_y = np.array(new_y)
        cMatrix = confusion_matrix(y_test[:,1].astype("int"), new_y)
        sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
        specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
        sen_test.append(sensivity)
        spec_test.append(specifity)
        fpr,tpr,thresholds = roc_curve(y_test[:,1], y_pred_log[:,1] )
        roc_auc = auc(fpr, tpr)
        roc_auc_test.append(roc_auc)
        f1=f1_score(y_test[:,1], new_y)
        f1_test.append(f1)
        ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
        npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

        ppv_test.append(ppv)
        npv_test.append(npv)

    mean_roc_auc_test.append(np.mean(roc_auc_test))
    mean_sen_test.append(np.mean(sen_test))
    mean_spec_test.append(np.mean(spec_test))
    mean_f1_test.append(np.mean(f1_test))
    mean_ppv_test.append(np.mean(ppv_test))
    mean_npv_test.append(np.mean(npv_test))

    model = LogisticRegression( max_iter=max_iter )

    np.random.seed(rm)
    model.fit(x, y[:,1])

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)
    cMatrix = confusion_matrix(y_validate[:,1].astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate[:,1], y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate[:,1], new_y)
    ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Mean area under the ROC curve = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_roc_auc_test),
                                                                       np.mean (mean_roc_auc_test) -
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test)) ,
                                                                       np.mean (mean_roc_auc_test) +
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test) )
                                                                             ))
print('Mean Sensitivity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen_test),
                                                                       np.mean (mean_sen_test) -
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test)) ,
                                                                       np.mean (mean_sen_test) +
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test))
                                                                               ))
print('Mean Specificity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec_test),
                                                                       np.mean (mean_spec_test) - 1.96 * np.std(mean_spec_test) /np.sqrt(len(mean_spec_test)),
                                                                       np.mean (mean_spec_test) + 1.96 * np.std(mean_spec_test)/np.sqrt(len(mean_spec_test) ))
                                                                            )
print('Mean F1-score ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1_test),
                                                                       np.mean (mean_f1_test) - 1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test)) ,
                                                                       np.mean (mean_f1_test) +
                                                                       1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test) ))
                                                                      )

print('Mean PPV ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv_test),
                                                                       np.mean (mean_ppv_test) -
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test)) ,
                                                                       np.mean (mean_ppv_test) +
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test) ))
                                                                      )

print('Mean NPV  ={:.4f} 95% [ {:.4f}, {:.4f}]'.format(np.mean (mean_npv_test),
                                                                   np.mean (mean_npv_test) -
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)),
                                                                   np.mean (mean_npv_test) +
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)))
                                                                  )

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )


###Model Maximum AUC (Table 4)

In [ ]:
name='cat Age'
data[name]=None
data.loc[(data['Age, years'] >=60 ),(name)]= 1
data.loc[(data['Age, years'] < 60 ),(name)]= 0

name='cat RAD'
data[name]=None
data.loc[(data['RAD, cm'] >= 4.12 ) , (name)]= 1
data.loc[(data['RAD, cm'] < 4.12 ) , (name)]= 0

name='cat LV ESD'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 3), (name)]= 1
data.loc[(data['LV ESD, cm'] < 3 ),(name)]= 0

name='cat QRS'
data[name]=None
data.loc[(data['QRS, ms\n'] < 81),(name)]= 1
data.loc[(data['QRS, ms\n'] >= 81 ),(name)]= 0

name='cat QT'
data[name]=None
data.loc[(data['QT, ms\n'] >= 382),(name)]= 1
data.loc[(data['QT, ms\n'] < 382 ),(name)]= 0

name='cat PQ'
data[name]=None
data.loc[(data['PQ, ms\n'] >= 163 ), (name)]= 1
data.loc[(data['PQ, ms\n'] < 163 ), (name)]= 0

name='cat RR'
data[name]=None
data.loc[ (data['RR, ms\n'] >= 882), (name)]= 1
data.loc[ (data['RR, ms\n'] < 882), (name)]= 0

name='cat P'
data[name]=None
data.loc[(data['P, ms'] >= 100),(name)]= 1
data.loc[(data['P, ms'] < 100 ),(name)]= 0


features =[ 'cat Age',  'cat RAD' , 'Tricuspid regurgitation' ,  'cat QRS' , 'cat QT', 'cat RR' ,'cat PQ', 'cat P' ,  'cat LV ESD'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.19
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
features.remove(y_name)
n_splits = 10

np.random.seed(rm)

########### LogisticRegression
max_iter = 100

mean_roc_auc_test = []
mean_sen_test = []
mean_spec_test = []
mean_f1_test = []
mean_ppv_test = []
mean_npv_test = []

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []


for j in range(100):
    rm=j+42
    np.random.seed(rm)
    # We set aside 20% for final testing
    x, x_validate, y, y_validate = train_test_split(      model_data[features],model_data[y_name],
                                                          train_size=0.8,
                                                          stratify=model_data[y_name],
                                                          random_state = rm )
    x = np.array(x)
    y = utils.to_categorical(y)
    x_validate = np.array(x_validate)
    y_validate = utils.to_categorical(y_validate)

    roc_auc_test=[]
    sen_test=[]
    spec_test=[]
    matr_test=[]
    f1_test=[]
    ppv_test=[]
    npv_test=[]
    threshold_test=[]
    skf = StratifiedKFold(n_splits=n_splits )

    # training and cross-validation
    for i, (train_index, test_index) in enumerate(skf.split(x, y[:,1])):

        x_train, x_test = x[train_index,:], x[test_index,:]
        y_train, y_test = y[train_index,:], y[test_index,:]

        model = LogisticRegression( max_iter=max_iter )

        np.random.seed(rm)
        model.fit(x_train, y_train[:,1])
        y_pred_log=model.predict_proba(x_test)
        new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
        new_y = np.array(new_y)
        cMatrix = confusion_matrix(y_test[:,1].astype("int"), new_y)
        sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
        specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
        sen_test.append(sensivity)
        spec_test.append(specifity)
        fpr,tpr,thresholds = roc_curve(y_test[:,1], y_pred_log[:,1] )
        roc_auc = auc(fpr, tpr)
        roc_auc_test.append(roc_auc)
        f1=f1_score(y_test[:,1], new_y)
        f1_test.append(f1)
        ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
        npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

        ppv_test.append(ppv)
        npv_test.append(npv)

    mean_roc_auc_test.append(np.mean(roc_auc_test))
    mean_sen_test.append(np.mean(sen_test))
    mean_spec_test.append(np.mean(spec_test))
    mean_f1_test.append(np.mean(f1_test))
    mean_ppv_test.append(np.mean(ppv_test))
    mean_npv_test.append(np.mean(npv_test))

    model = LogisticRegression( max_iter=max_iter )

    np.random.seed(rm)
    model.fit(x, y[:,1])

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)
    cMatrix = confusion_matrix(y_validate[:,1].astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate[:,1], y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate[:,1], new_y)
    ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Mean area under the ROC curve = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_roc_auc_test),
                                                                       np.mean (mean_roc_auc_test) -
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test)) ,
                                                                       np.mean (mean_roc_auc_test) +
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test) )
                                                                             ))
print('Mean Sensitivity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen_test),
                                                                       np.mean (mean_sen_test) -
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test)) ,
                                                                       np.mean (mean_sen_test) +
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test))
                                                                               ))
print('Mean Specificity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec_test),
                                                                       np.mean (mean_spec_test) - 1.96 * np.std(mean_spec_test) /np.sqrt(len(mean_spec_test)),
                                                                       np.mean (mean_spec_test) + 1.96 * np.std(mean_spec_test)/np.sqrt(len(mean_spec_test) ))
                                                                            )
print('Mean F1-score ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1_test),
                                                                       np.mean (mean_f1_test) - 1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test)) ,
                                                                       np.mean (mean_f1_test) +
                                                                       1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test) ))
                                                                      )

print('Mean PPV ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv_test),
                                                                       np.mean (mean_ppv_test) -
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test)) ,
                                                                       np.mean (mean_ppv_test) +
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test) ))
                                                                      )

print('Mean NPV  ={:.4f} 95% [ {:.4f}, {:.4f}]'.format(np.mean (mean_npv_test),
                                                                   np.mean (mean_npv_test) -
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)),
                                                                   np.mean (mean_npv_test) +
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)))
                                                                  )

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )


###Model Centroid (Table 4)



In [ ]:
name='cat Age'
data[name]=None
data.loc[(data['Age, years'] >=64 ),(name)]= 1
data.loc[(data['Age, years'] < 64 ),(name)]= 0

name='cat RAD'
data[name]=None
data.loc[(data['RAD, cm'] >= 3.33 ) , (name)]= 1
data.loc[(data['RAD, cm'] < 3.33 ) , (name)]= 0

name='cat LV ESD'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 4.4), (name)]= 1
data.loc[(data['LV ESD, cm'] < 4.4 ),(name)]= 0

name='cat QRS'
data[name]=None
data.loc[(data['QRS, ms\n'] < 80),(name)]= 1
data.loc[(data['QRS, ms\n'] >= 80 ),(name)]= 0

name='cat QT'
data[name]=None
data.loc[(data['QT, ms\n'] >= 390),(name)]= 1
data.loc[(data['QT, ms\n'] < 390 ),(name)]= 0

name='cat PQ'
data[name]=None
data.loc[(data['PQ, ms\n'] >= 155 ), (name)]= 1
data.loc[(data['PQ, ms\n'] < 155 ), (name)]= 0

name='cat RR'
data[name]=None
data.loc[ (data['RR, ms\n'] >= 925), (name)]= 1
data.loc[ (data['RR, ms\n'] < 925), (name)]= 0

name='cat P'
data[name]=None
data.loc[(data['P, ms'] >= 100),(name)]= 1
data.loc[(data['P, ms'] < 100 ),(name)]= 0


features =[ 'cat Age',  'cat RAD' , 'Tricuspid regurgitation' ,  'cat QRS' , 'cat QT', 'cat RR' ,'cat PQ', 'cat P' ,  'cat LV ESD'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.185
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
features.remove(y_name)
n_splits = 10

np.random.seed(rm)

########### LogisticRegression
max_iter=100

mean_roc_auc_test = []
mean_sen_test = []
mean_spec_test = []
mean_f1_test = []
mean_ppv_test = []
mean_npv_test = []

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []


for j in range(100):
    rm=j+42
    np.random.seed(rm)
    # We set aside 20% for final testing
    x, x_validate, y, y_validate = train_test_split(      model_data[features],model_data[y_name],
                                                          train_size=0.8,
                                                          stratify=model_data[y_name],
                                                          random_state = rm )
    x = np.array(x)
    y = utils.to_categorical(y)
    x_validate = np.array(x_validate)
    y_validate = utils.to_categorical(y_validate)

    roc_auc_test=[]
    sen_test=[]
    spec_test=[]
    matr_test=[]
    f1_test=[]
    ppv_test=[]
    npv_test=[]
    threshold_test=[]
    skf = StratifiedKFold(n_splits=n_splits )

    # training and cross-validation
    for i, (train_index, test_index) in enumerate(skf.split(x, y[:,1])):

        x_train, x_test = x[train_index,:], x[test_index,:]
        y_train, y_test = y[train_index,:], y[test_index,:]

        model = LogisticRegression( max_iter=max_iter )

        np.random.seed(rm)
        model.fit(x_train, y_train[:,1])
        y_pred_log=model.predict_proba(x_test)
        new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
        new_y = np.array(new_y)
        cMatrix = confusion_matrix(y_test[:,1].astype("int"), new_y)
        sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
        specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
        sen_test.append(sensivity)
        spec_test.append(specifity)
        fpr,tpr,thresholds = roc_curve(y_test[:,1], y_pred_log[:,1] )
        roc_auc = auc(fpr, tpr)
        roc_auc_test.append(roc_auc)
        f1=f1_score(y_test[:,1], new_y)
        f1_test.append(f1)
        ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
        npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

        ppv_test.append(ppv)
        npv_test.append(npv)

    mean_roc_auc_test.append(np.mean(roc_auc_test))
    mean_sen_test.append(np.mean(sen_test))
    mean_spec_test.append(np.mean(spec_test))
    mean_f1_test.append(np.mean(f1_test))
    mean_ppv_test.append(np.mean(ppv_test))
    mean_npv_test.append(np.mean(npv_test))

    model = LogisticRegression( max_iter=max_iter )

    np.random.seed(rm)
    model.fit(x, y[:,1])

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)
    cMatrix = confusion_matrix(y_validate[:,1].astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate[:,1], y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate[:,1], new_y)
    ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    if (cMatrix[0][0] + cMatrix[1][0] ) > 0:
      npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Mean area under the ROC curve = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_roc_auc_test),
                                                                       np.mean (mean_roc_auc_test) -
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test)) ,
                                                                       np.mean (mean_roc_auc_test) +
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test) )
                                                                             ))
print('Mean Sensitivity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen_test),
                                                                       np.mean (mean_sen_test) -
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test)) ,
                                                                       np.mean (mean_sen_test) +
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test))
                                                                               ))
print('Mean Specificity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec_test),
                                                                       np.mean (mean_spec_test) - 1.96 * np.std(mean_spec_test) /np.sqrt(len(mean_spec_test)),
                                                                       np.mean (mean_spec_test) + 1.96 * np.std(mean_spec_test)/np.sqrt(len(mean_spec_test) ))
                                                                            )
print('Mean F1-score ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1_test),
                                                                       np.mean (mean_f1_test) - 1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test)) ,
                                                                       np.mean (mean_f1_test) +
                                                                       1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test) ))
                                                                      )

print('Mean PPV ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv_test),
                                                                       np.mean (mean_ppv_test) -
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test)) ,
                                                                       np.mean (mean_ppv_test) +
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test) ))
                                                                      )

print('Mean NPV  ={:.4f} 95% [ {:.4f}, {:.4f}]'.format(np.mean (mean_npv_test),
                                                                   np.mean (mean_npv_test) -
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)),
                                                                   np.mean (mean_npv_test) +
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)))
                                                                  )

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )


###Model SHAP (Table 4)

In [ ]:
name='cat Age'
data[name]=None
data.loc[(data['Age, years'] >=61 ),(name)]= 1
data.loc[(data['Age, years'] < 61 ),(name)]= 0

name='cat LV ESD 1'
data[name]=None
data.loc[( (data['LV ESD, cm'] >= 3.1) & ((data['LV ESD, cm'] < 4.1)) ), (name)]= 1
data.loc[( (data['LV ESD, cm'] < 3.1) | ((data['LV ESD, cm'] >= 4.1)) ), (name)]= 0

name='cat LV ESD 2'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 5), (name)]= 1
data.loc[(data['LV ESD, cm'] < 5 ),(name)]= 0

name='cat RAD'
data[name]=None
data.loc[( (data['RAD, cm'] >= 4.2) & ((data['RAD, cm'] < 5.3)) ), (name)]= 1
data.loc[( (data['RAD, cm'] < 4.2) | ((data['RAD, cm'] >= 5.3)) ), (name)]= 0

name='cat QRS'
data[name]=None
data.loc[(data['QRS, ms\n'] >= 80),(name)]= 1
data.loc[(data['QRS, ms\n'] < 80 ),(name)]= 0

name='cat QT'
data[name]=None
data.loc[(data['QT, ms\n'] >= 390),(name)]= 1
data.loc[(data['QT, ms\n'] < 390 ),(name)]= 0

name='cat PQ'
data[name]=None
data.loc[( (data['PQ, ms\n'] >= 170) & ((data['PQ, ms\n'] < 210)) ), (name)]= 1
data.loc[( (data['PQ, ms\n'] < 170) | ((data['PQ, ms\n'] >= 210)) ), (name)]= 0

name='cat RR 1'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 700) & ((data['RR, ms\n'] < 750)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 700) | ((data['RR, ms\n'] >= 750)) ), (name)]= 0

name='cat RR 2'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 880) & ((data['RR, ms\n'] < 1000)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 880) | ((data['RR, ms\n'] >= 1000)) ), (name)]= 0

name='cat RR 3'
data[name]=None
data.loc[ (data['RR, ms\n'] >= 1100), (name)]= 1
data.loc[ (data['RR, ms\n'] < 1100), (name)]= 0

name='cat P'
data[name]=None
data.loc[(data['P, ms'] >= 130),(name)]= 1
data.loc[(data['P, ms'] < 130 ),(name)]= 0


features =[ 'cat Age',  'cat RAD', 'cat LV ESD 1', "cat LV ESD 2", 'cat QRS' , 'cat QT',
            'cat RR 1', 'cat RR 2', 'cat RR 3', 'cat PQ', 'cat P', 'Tricuspid regurgitation'
          ]
rm = 100
border = 0.18
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
features.remove(y_name)
n_splits = 10

np.random.seed(rm)

########### LogisticRegression
max_iter=100

mean_roc_auc_test = []
mean_sen_test = []
mean_spec_test = []
mean_f1_test = []
mean_ppv_test = []
mean_npv_test = []

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []


for j in range(100):
    rm=j+42
    np.random.seed(rm)
    # We set aside 20% for final testing
    x, x_validate, y, y_validate = train_test_split(      model_data[features],model_data[y_name],
                                                          train_size=0.8,
                                                          stratify=model_data[y_name],
                                                          random_state = rm )
    x = np.array(x)
    y = utils.to_categorical(y)
    x_validate = np.array(x_validate)
    y_validate = utils.to_categorical(y_validate)

    roc_auc_test=[]
    sen_test=[]
    spec_test=[]
    matr_test=[]
    f1_test=[]
    ppv_test=[]
    npv_test=[]
    threshold_test=[]
    skf = StratifiedKFold(n_splits=n_splits )

    # training and cross-validation
    for i, (train_index, test_index) in enumerate(skf.split(x, y[:,1])):

        x_train, x_test = x[train_index,:], x[test_index,:]
        y_train, y_test = y[train_index,:], y[test_index,:]

        model = LogisticRegression( max_iter=max_iter )

        np.random.seed(rm)
        model.fit(x_train, y_train[:,1])
        y_pred_log=model.predict_proba(x_test)
        new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
        new_y = np.array(new_y)
        cMatrix = confusion_matrix(y_test[:,1].astype("int"), new_y)
        sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
        specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
        sen_test.append(sensivity)
        spec_test.append(specifity)
        fpr,tpr,thresholds = roc_curve(y_test[:,1], y_pred_log[:,1] )
        roc_auc = auc(fpr, tpr)
        roc_auc_test.append(roc_auc)
        f1=f1_score(y_test[:,1], new_y)
        f1_test.append(f1)
        ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
        if (cMatrix[0][0] + cMatrix[1][0] ) > 0:
          npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

        ppv_test.append(ppv)
        npv_test.append(npv)

    mean_roc_auc_test.append(np.mean(roc_auc_test))
    mean_sen_test.append(np.mean(sen_test))
    mean_spec_test.append(np.mean(spec_test))
    mean_f1_test.append(np.mean(f1_test))
    mean_ppv_test.append(np.mean(ppv_test))
    mean_npv_test.append(np.mean(npv_test))

    model = LogisticRegression( max_iter=max_iter )

    np.random.seed(rm)
    model.fit(x, y[:,1])

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)
    cMatrix = confusion_matrix(y_validate[:,1].astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate[:,1], y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate[:,1], new_y)
    ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    if (cMatrix[0][0] + cMatrix[1][0] ) > 0:
      npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Mean area under the ROC curve = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_roc_auc_test),
                                                                       np.mean (mean_roc_auc_test) -
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test)) ,
                                                                       np.mean (mean_roc_auc_test) +
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test) )
                                                                             ))
print('Mean Sensitivity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen_test),
                                                                       np.mean (mean_sen_test) -
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test)) ,
                                                                       np.mean (mean_sen_test) +
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test))
                                                                               ))
print('Mean Specificity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec_test),
                                                                       np.mean (mean_spec_test) - 1.96 * np.std(mean_spec_test) /np.sqrt(len(mean_spec_test)),
                                                                       np.mean (mean_spec_test) + 1.96 * np.std(mean_spec_test)/np.sqrt(len(mean_spec_test) ))
                                                                            )
print('Mean F1-score ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1_test),
                                                                       np.mean (mean_f1_test) - 1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test)) ,
                                                                       np.mean (mean_f1_test) +
                                                                       1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test) ))
                                                                      )

print('Mean PPV ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv_test),
                                                                       np.mean (mean_ppv_test) -
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test)) ,
                                                                       np.mean (mean_ppv_test) +
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test) ))
                                                                      )

print('Mean NPV  ={:.4f} 95% [ {:.4f}, {:.4f}]'.format(np.mean (mean_npv_test),
                                                                   np.mean (mean_npv_test) -
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)),
                                                                   np.mean (mean_npv_test) +
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)))
                                                                  )

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )


##Predictor weights (Table 5)

###SHAP cutoff thresholds

In [24]:
name='cat Age'
data[name]=None
data.loc[(data['Age, years'] >=61 ),(name)]= 1
data.loc[(data['Age, years'] < 61 ),(name)]= 0

name='cat LV ESD 1'
data[name]=None
data.loc[( (data['LV ESD, cm'] >= 3.1) & ((data['LV ESD, cm'] < 4.1)) ), (name)]= 1
data.loc[( (data['LV ESD, cm'] < 3.1) | ((data['LV ESD, cm'] >= 4.1)) ), (name)]= 0

name='cat LV ESD 2'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 5), (name)]= 1
data.loc[(data['LV ESD, cm'] < 5 ),(name)]= 0

name='cat RAD'
data[name]=None
data.loc[( (data['RAD, cm'] >= 4.2) & ((data['RAD, cm'] < 5.3)) ), (name)]= 1
data.loc[( (data['RAD, cm'] < 4.2) | ((data['RAD, cm'] >= 5.3)) ), (name)]= 0

name='cat QRS'
data[name]=None
data.loc[(data['QRS, ms\n'] >= 80),(name)]= 1
data.loc[(data['QRS, ms\n'] < 80 ),(name)]= 0

name='cat QT'
data[name]=None
data.loc[(data['QT, ms\n'] >= 390),(name)]= 1
data.loc[(data['QT, ms\n'] < 390 ),(name)]= 0

name='cat PQ'
data[name]=None
data.loc[( (data['PQ, ms\n'] >= 170) & ((data['PQ, ms\n'] < 210)) ), (name)]= 1
data.loc[( (data['PQ, ms\n'] < 170) | ((data['PQ, ms\n'] >= 210)) ), (name)]= 0

name='cat RR 1'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 700) & ((data['RR, ms\n'] < 750)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 700) | ((data['RR, ms\n'] >= 750)) ), (name)]= 0

name='cat RR 2'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 880) & ((data['RR, ms\n'] < 1100)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 880) | ((data['RR, ms\n'] >= 1100)) ), (name)]= 0

name='cat RR 3'
data[name]=None
data.loc[ (data['RR, ms\n'] >= 1100), (name)]= 1
data.loc[ (data['RR, ms\n'] < 1100), (name)]= 0

name='cat P'
data[name]=None
data.loc[(data['P, ms'] >= 130),(name)]= 1
data.loc[(data['P, ms'] < 130 ),(name)]= 0


features =[ 'cat Age',  'cat RAD', 'cat LV ESD 1', "cat LV ESD 2", 'cat QRS' , 'cat QT',
            'cat RR 1', 'cat RR 2', 'cat RR 3', 'cat PQ', 'cat P', 'Tricuspid regurgitation'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.2
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
features.remove(y_name)
n_splits = 10

np.random.seed(rm)

logistReg = LogisticRegression()
logistReg.fit( model_data[features].values, model_data[y_name].values )

for i in range(0, len(logistReg.coef_[0])):
  print( "Feature: " + features[i] + ", Weight: " + str( logistReg.coef_[0][i] ) )


Feature: cat Age, Weight: 0.6832867581742813
Feature: cat RAD, Weight: 0.9170410657376464
Feature: cat LV ESD 1, Weight: 0.9588301080327979
Feature: cat LV ESD 2, Weight: 1.1545495548896885
Feature: cat QRS, Weight: 0.25506632493746134
Feature: cat QT, Weight: 0.5420264924976407
Feature: cat RR 1, Weight: 1.1451430255750896
Feature: cat RR 2, Weight: 0.911753615488855
Feature: cat RR 3, Weight: 0.740143562159023
Feature: cat PQ, Weight: 0.9803811120776253
Feature: cat P, Weight: 1.33451060274336
Feature: Tricuspid regurgitation, Weight: 0.7347620998913463


###Multimetric categorization cutoff thresholds

In [ ]:
name='cat Age'
data[name]=None
data.loc[(data['Age, years'] >=60 ),(name)]= 1
data.loc[(data['Age, years'] < 60 ),(name)]= 0

name='cat LV ESD 1'
data[name]=None
data.loc[( (data['LV ESD, cm'] >= 3.1) & ((data['LV ESD, cm'] < 4.1)) ), (name)]= 1
data.loc[( (data['LV ESD, cm'] < 3.1) | ((data['LV ESD, cm'] >= 4.1)) ), (name)]= 0

name='cat LV ESD 2'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 5), (name)]= 1
data.loc[(data['LV ESD, cm'] < 5 ),(name)]= 0

name='cat RAD'
data[name]=None
data.loc[( (data['RAD, cm'] >= 4.2) & ((data['RAD, cm'] < 5.3)) ), (name)]= 1
data.loc[( (data['RAD, cm'] < 4.2) | ((data['RAD, cm'] >= 5.3)) ), (name)]= 0

name='cat QRS'
data[name]=None
data.loc[(data['QRS, ms\n'] >= 80),(name)]= 1
data.loc[(data['QRS, ms\n'] < 80 ),(name)]= 0

name='cat QT'
data[name]=None
data.loc[(data['QT, ms\n'] >= 390),(name)]= 1
data.loc[(data['QT, ms\n'] < 390 ),(name)]= 0

name='cat PQ'
data[name]=None
data.loc[( (data['PQ, ms\n'] >= 170) & ((data['PQ, ms\n'] < 210)) ), (name)]= 1
data.loc[( (data['PQ, ms\n'] < 170) | ((data['PQ, ms\n'] >= 210)) ), (name)]= 0

name='cat RR 1'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 700) & ((data['RR, ms\n'] < 750)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 700) | ((data['RR, ms\n'] >= 750)) ), (name)]= 0

name='cat RR 2'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 880) & ((data['RR, ms\n'] < 1100)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 880) | ((data['RR, ms\n'] >= 1100)) ), (name)]= 0

name='cat RR 3'
data[name]=None
data.loc[ (data['RR, ms\n'] >= 1100), (name)]= 1
data.loc[ (data['RR, ms\n'] < 1100), (name)]= 0

name='cat P 1'
data[name]=None
data.loc[( (data['P, ms'] >= 100) & ((data['P, ms'] < 130)) ), (name)]= 1
data.loc[( (data['P, ms'] < 100) | ((data['P, ms'] >= 130)) ), (name)]= 0

name='cat P 2'
data[name]=None
data.loc[(data['P, ms'] >= 130),(name)]= 1
data.loc[(data['P, ms'] < 130 ),(name)]= 0


features =[ 'cat Age',  'cat RAD', 'cat LV ESD 1', "cat LV ESD 2", 'cat QRS' , 'cat QT',
            'cat RR 1', 'cat RR 2', 'cat RR 3', 'cat PQ', 'cat P 1', 'cat P 2', 'Tricuspid regurgitation'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.2
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
features.remove(y_name)
n_splits = 10

np.random.seed(rm)

logistReg = LogisticRegression()
logistReg.fit( model_data[features].values, model_data[y_name].values )

for i in range(0, len(logistReg.coef_[0])):
  print( "Feature: " + features[i] + ", Weight: " + str( logistReg.coef_[0][i] ) )

###Medians of the groups and the centroid cutoff thresholds

In [26]:
name='cat Age 1'
data[name]=None
data.loc[( (data['Age, years'] >= 63) & ((data['Age, years'] < 66)) ), (name)]= 1
data.loc[( (data['Age, years'] < 63) | ((data['Age, years'] >= 66)) ), (name)]= 0
name='cat Age 2'
data[name]=None
data.loc[(data['Age, years'] >=66 ),(name)]= 1
data.loc[(data['Age, years'] < 66 ),(name)]= 0

name='cat LV ESD 1'
data[name]=None
data.loc[( (data['LV ESD, cm'] >= 3.3) & ((data['LV ESD, cm'] < 3.33)) ), (name)]= 1
data.loc[( (data['LV ESD, cm'] < 3.3) | ((data['LV ESD, cm'] >= 3.33)) ), (name)]= 0
name='cat LV ESD 2'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 3.33), (name)]= 1
data.loc[(data['LV ESD, cm'] < 3.33 ),(name)]= 0


name='cat RAD 1'
data[name]=None
data.loc[( (data['RAD, cm'] >= 4.4) & ((data['RAD, cm'] < 4.5)) ), (name)]= 1
data.loc[( (data['RAD, cm'] < 4.4) | ((data['RAD, cm'] >= 4.5)) ), (name)]= 0

name='cat QRS'
data[name]=None
data.loc[(data['QRS, ms\n'] >= 80),(name)]= 1
data.loc[(data['QRS, ms\n'] < 80 ),(name)]= 0


name='cat QT 1'
data[name]=None
data.loc[( (data['QT, ms\n'] >= 380) & ((data['QT, ms\n'] < 400)) ), (name)]= 1
data.loc[( (data['QT, ms\n'] < 380) | ((data['QT, ms\n'] >= 400)) ), (name)]= 0
name='cat QT 2'
data[name]=None
data.loc[(data['QT, ms\n'] >= 400),(name)]= 1
data.loc[(data['QT, ms\n'] < 400 ),(name)]= 0


name='cat PQ'
data[name]=None
data.loc[(data['PQ, ms\n'] >= 155),(name)]= 1
data.loc[(data['PQ, ms\n'] < 155 ),(name)]= 0


name='cat RR 1'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 925) & ((data['RR, ms\n'] < 950)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 925) | ((data['RR, ms\n'] >= 950)) ), (name)]= 0
name='cat RR 2'
data[name]=None
data.loc[(data['RR, ms\n'] >= 950),(name)]= 1
data.loc[(data['RR, ms\n'] < 950),(name)]= 0

name='cat P'
data[name]=None
data.loc[(data['P, ms'] >= 100),(name)]= 1
data.loc[(data['P, ms'] < 100),(name)]= 0


features =[ 'cat Age 1', 'cat Age 2',
            'cat LV ESD 1', 'cat LV ESD 2', "cat RAD 1",  'cat QRS',
            'cat QT 1', 'cat QT 2', 'cat PQ', 'cat RR 2', 'cat P', 'Tricuspid regurgitation'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.2
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
features.remove(y_name)
n_splits = 10

np.random.seed(rm)

logistReg = LogisticRegression()
logistReg.fit( model_data[features].values, model_data[y_name].values )

for i in range(0, len(features)):
  print( "Feature: " + features[i] + ", Weight: " + str( logistReg.coef_[0][i] ) )

Feature: cat Age 1, Weight: 0.3969683346816919
Feature: cat Age 2, Weight: 0.31724870109528286
Feature: cat LV ESD 1, Weight: 0.6739879567045161
Feature: cat LV ESD 2, Weight: 0.1803738942876293
Feature: cat RAD 1, Weight: 0.361449017709421
Feature: cat QRS, Weight: 0.3817288777289737
Feature: cat QT 1, Weight: 0.17361282790739463
Feature: cat QT 2, Weight: 0.5905688027144186
Feature: cat PQ, Weight: 0.5965503502958382
Feature: cat RR 2, Weight: 0.21964849836934633
Feature: cat P, Weight: 1.2808633877782962
Feature: Tricuspid regurgitation, Weight: 0.6842866175393167


###Quartiles cutoff thresholds

In [ ]:
name='cat Age 1'
data[name]=None
data.loc[( (data['Age, years'] >= 58) & ((data['Age, years'] < 64)) ), (name)]= 1
data.loc[( (data['Age, years'] < 58) | ((data['Age, years'] >= 64)) ), (name)]= 0
name='cat Age 2'
data[name]=None
data.loc[( (data['Age, years'] >= 64) & ((data['Age, years'] < 69)) ), (name)]= 1
data.loc[( (data['Age, years'] < 64) | ((data['Age, years'] >= 69)) ), (name)]= 0
name='cat Age 3'
data[name]=None
data.loc[(data['Age, years'] >=69 ),(name)]= 1
data.loc[(data['Age, years'] < 69 ),(name)]= 0


name='cat LV ESD 1'
data[name]=None
data.loc[( (data['LV ESD, cm'] >= 3.1) & ((data['LV ESD, cm'] < 3.3)) ), (name)]= 1
data.loc[( (data['LV ESD, cm'] < 3.1) | ((data['LV ESD, cm'] >= 3.3)) ), (name)]= 0
name='cat LV ESD 2'
data[name]=None
data.loc[( (data['LV ESD, cm'] >= 3.3) & ((data['LV ESD, cm'] < 3.8)) ), (name)]= 1
data.loc[( (data['LV ESD, cm'] < 3.3) | ((data['LV ESD, cm'] >= 3.8)) ), (name)]= 0
name='cat LV ESD 3'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 3.8), (name)]= 1
data.loc[(data['LV ESD, cm'] < 3.8 ),(name)]= 0


name='cat RAD 1'
data[name]=None
data.loc[( (data['RAD, cm'] >= 3.8) & ((data['RAD, cm'] < 4.3)) ), (name)]= 1
data.loc[( (data['RAD, cm'] < 3.8) | ((data['RAD, cm'] >= 4.3)) ), (name)]= 0
name='cat RAD 2'
data[name]=None
data.loc[( (data['RAD, cm'] >= 4.3) & ((data['RAD, cm'] < 4.8)) ), (name)]= 1
data.loc[( (data['RAD, cm'] < 4.3) | ((data['RAD, cm'] >= 4.8)) ), (name)]= 0
name='cat RAD 3'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 4.8), (name)]= 1
data.loc[(data['LV ESD, cm'] < 4.8 ),(name)]= 0


name='cat QRS'
data[name]=None
data.loc[( (data['QRS, ms\n'] >= 80) & ((data['QRS, ms\n'] < 100)) ), (name)]= 1
data.loc[( (data['QRS, ms\n'] < 80) | ((data['QRS, ms\n'] >= 100)) ), (name)]= 0


name='cat QT 1'
data[name]=None
data.loc[( (data['QT, ms\n'] >= 360) & ((data['QT, ms\n'] < 395)) ), (name)]= 1
data.loc[( (data['QT, ms\n'] < 360) | ((data['QT, ms\n'] >= 395)) ), (name)]= 0
name='cat QT 2'
data[name]=None
data.loc[( (data['QT, ms\n'] >= 395) & ((data['QT, ms\n'] < 420)) ), (name)]= 1
data.loc[( (data['QT, ms\n'] < 395) | ((data['QT, ms\n'] >= 420)) ), (name)]= 0
name='cat QT 3'
data[name]=None
data.loc[(data['QT, ms\n'] >= 420),(name)]= 1
data.loc[(data['QT, ms\n'] < 420 ),(name)]= 0


name='cat RR 1'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 800) & ((data['RR, ms\n'] < 920)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 800) | ((data['RR, ms\n'] >= 920)) ), (name)]= 0
name='cat RR 2'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 920) & ((data['RR, ms\n'] < 1080)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 920) | ((data['RR, ms\n'] >= 1080)) ), (name)]= 0
name='cat RR 3'
data[name]=None
data.loc[(data['RR, ms\n'] >= 1080),(name)]= 1
data.loc[(data['RR, ms\n'] < 1080),(name)]= 0


name='cat PQ 1'
data[name]=None
data.loc[( (data['PQ, ms\n'] >= 140) & ((data['PQ, ms\n'] < 160)) ), (name)]= 1
data.loc[( (data['PQ, ms\n'] < 140) | ((data['PQ, ms\n'] >= 160)) ), (name)]= 0
name='cat PQ 2'
data[name]=None
data.loc[( (data['PQ, ms\n'] >= 160) & ((data['PQ, ms\n'] < 180)) ), (name)]= 1
data.loc[( (data['PQ, ms\n'] < 160) | ((data['PQ, ms\n'] >= 180)) ), (name)]= 0
name='cat PQ 3'
data[name]=None
data.loc[(data['PQ, ms\n'] >= 180),(name)]= 1
data.loc[(data['PQ, ms\n'] < 180 ),(name)]= 0


name='cat P'
data[name]=None
data.loc[(data['P, ms'] >= 100),(name)]= 1
data.loc[(data['P, ms'] < 100),(name)]= 0


features =[ 'cat Age 1', 'cat Age 2', 'cat Age 3',
            'cat LV ESD 1', 'cat LV ESD 2', 'cat LV ESD 3', "cat RAD 1", 'cat RAD 2', 'cat QRS', 'cat QT 2', 'cat QT 3',
            'cat RR 2', 'cat RR 3', 'cat PQ 2', 'cat PQ 3', 'cat P', 'Tricuspid regurgitation'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.2
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
features.remove(y_name)
n_splits = 10

np.random.seed(rm)

logistReg = LogisticRegression()
logistReg.fit( model_data[features].values, model_data[y_name].values )

for i in range(0, len(features)):
  print( "Feature: " + features[i] + ", Weight: " + str( logistReg.coef_[0][i] ) )

##Development of models based on multi-level variables (Table 6)

###Model SHAP

In [ ]:
name='cat Age'
data[name]=None
data.loc[(data['Age, years'] >=61 ),(name)]= 1
data.loc[(data['Age, years'] < 61 ),(name)]= 0

name='cat LV ESD 1'
data[name]=None
data.loc[( (data['LV ESD, cm'] >= 3.1) & ((data['LV ESD, cm'] < 4.1)) ), (name)]= 1
data.loc[( (data['LV ESD, cm'] < 3.1) | ((data['LV ESD, cm'] >= 4.1)) ), (name)]= 0

name='cat LV ESD 2'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 5), (name)]= 1
data.loc[(data['LV ESD, cm'] < 5 ),(name)]= 0

name='cat RAD'
data[name]=None
data.loc[( (data['RAD, cm'] >= 4.2) & ((data['RAD, cm'] < 5.3)) ), (name)]= 1
data.loc[( (data['RAD, cm'] < 4.2) | ((data['RAD, cm'] >= 5.3)) ), (name)]= 0

name='cat QRS'
data[name]=None
data.loc[(data['QRS, ms\n'] >= 80),(name)]= 1
data.loc[(data['QRS, ms\n'] < 80 ),(name)]= 0

name='cat QT'
data[name]=None
data.loc[(data['QT, ms\n'] >= 390),(name)]= 1
data.loc[(data['QT, ms\n'] < 390 ),(name)]= 0

name='cat PQ'
data[name]=None
data.loc[( (data['PQ, ms\n'] >= 170) & ((data['PQ, ms\n'] < 210)) ), (name)]= 1
data.loc[( (data['PQ, ms\n'] < 170) | ((data['PQ, ms\n'] >= 210)) ), (name)]= 0

name='cat RR 1'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 700) & ((data['RR, ms\n'] < 750)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 700) | ((data['RR, ms\n'] >= 750)) ), (name)]= 0

name='cat RR 2'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 880) & ((data['RR, ms\n'] < 1100)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 880) | ((data['RR, ms\n'] >= 1100)) ), (name)]= 0

name='cat RR 3'
data[name]=None
data.loc[ (data['RR, ms\n'] >= 1100), (name)]= 1
data.loc[ (data['RR, ms\n'] < 1100), (name)]= 0

name='cat P'
data[name]=None
data.loc[(data['P, ms'] >= 130),(name)]= 1
data.loc[(data['P, ms'] < 130 ),(name)]= 0


features =[ 'cat Age',  'cat RAD', 'cat LV ESD 1', "cat LV ESD 2", 'cat QRS' , 'cat QT',
            'cat RR 1', 'cat RR 2', 'cat RR 3', 'cat PQ', 'cat P', 'Tricuspid regurgitation'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.42
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
features.remove(y_name)
n_splits = 10

########### XGBoost
lr=0.03
m_d=2
n_e=200
spw=3
child_weight=1
sub=0.7
max_step=1
gam=2
method ='exact'
boost= 'gbtree'

mean_roc_auc_test = []
mean_sen_test = []
mean_spec_test = []
mean_f1_test = []
mean_ppv_test = []
mean_npv_test = []

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []


for j in range(100):
    rm=j+42
    np.random.seed(rm)
    # We set aside 20% for final testing
    x, x_validate, y, y_validate = train_test_split(      model_data[features],model_data[y_name],
                                                          train_size=0.8,
                                                          stratify=model_data[y_name],
                                                          random_state = rm )
    x = np.array(x)
    y = utils.to_categorical(y)
    x_validate = np.array(x_validate)
    y_validate = utils.to_categorical(y_validate)

    roc_auc_test=[]
    sen_test=[]
    spec_test=[]
    matr_test=[]
    f1_test=[]
    ppv_test=[]
    npv_test=[]
    threshold_test=[]
    skf = StratifiedKFold(n_splits=n_splits )

    # training and cross-validation
    for i, (train_index, test_index) in enumerate(skf.split(x, y[:,1])):

        x_train, x_test = x[train_index,:], x[test_index,:]
        y_train, y_test = y[train_index,:], y[test_index,:]

        model = xgb.XGBClassifier(learning_rate=lr,  eval_metric = "auc",
                                      scale_pos_weight = spw,    max_depth=m_d,  n_estimators=n_e,  random_state=rm,
                                      verbosity=0 , objective= 'binary:logistic', booster= boost,
                                      tree_method =method, max_delta_step=max_step, gamma=gam,
                                      min_child_weight=child_weight,
                                      subsample=sub, colsample_bylevel=1,
                                      n_jobs=-1, use_label_encoder=False
                                     )

        np.random.seed(rm)
        model.fit(x_train, y_train[:,1])
        y_pred_log=model.predict_proba(x_test)
        new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
        new_y = np.array(new_y)
        cMatrix = confusion_matrix(y_test[:,1].astype("int"), new_y)
        sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
        specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
        sen_test.append(sensivity)
        spec_test.append(specifity)
        fpr,tpr,thresholds = roc_curve(y_test[:,1], y_pred_log[:,1] )
        roc_auc = auc(fpr, tpr)
        roc_auc_test.append(roc_auc)
        f1=f1_score(y_test[:,1], new_y)
        f1_test.append(f1)
        ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
        npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

        ppv_test.append(ppv)
        npv_test.append(npv)

    mean_roc_auc_test.append(np.mean(roc_auc_test))
    mean_sen_test.append(np.mean(sen_test))
    mean_spec_test.append(np.mean(spec_test))
    mean_f1_test.append(np.mean(f1_test))
    mean_ppv_test.append(np.mean(ppv_test))
    mean_npv_test.append(np.mean(npv_test))

    model = xgb.XGBClassifier(learning_rate=lr,  eval_metric = "auc",
                                      scale_pos_weight = spw,    max_depth=m_d,  n_estimators=n_e,  random_state=rm,
                                      verbosity=0 , objective= 'binary:logistic', booster= boost,
                                      tree_method =method, max_delta_step=max_step, gamma=gam,
                                      min_child_weight=child_weight,
                                      subsample=sub, colsample_bylevel=1,
                                      n_jobs=-1, use_label_encoder=False
                                     )

    np.random.seed(rm)
    model.fit(x, y[:,1])

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)
    cMatrix = confusion_matrix(y_validate[:,1].astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate[:,1], y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate[:,1], new_y)
    ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Mean area under the ROC curve = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_roc_auc_test),
                                                                       np.mean (mean_roc_auc_test) -
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test)) ,
                                                                       np.mean (mean_roc_auc_test) +
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test) )
                                                                             ))
print('Mean Sensitivity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen_test),
                                                                       np.mean (mean_sen_test) -
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test)) ,
                                                                       np.mean (mean_sen_test) +
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test))
                                                                               ))
print('Mean Specificity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec_test),
                                                                       np.mean (mean_spec_test) - 1.96 * np.std(mean_spec_test) /np.sqrt(len(mean_spec_test)),
                                                                       np.mean (mean_spec_test) + 1.96 * np.std(mean_spec_test)/np.sqrt(len(mean_spec_test) ))
                                                                            )
print('Mean F1-score ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1_test),
                                                                       np.mean (mean_f1_test) - 1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test)) ,
                                                                       np.mean (mean_f1_test) +
                                                                       1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test) ))
                                                                      )

print('Mean PPV ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv_test),
                                                                       np.mean (mean_ppv_test) -
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test)) ,
                                                                       np.mean (mean_ppv_test) +
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test) ))
                                                                      )

print('Mean NPV  ={:.4f} 95% [ {:.4f}, {:.4f}]'.format(np.mean (mean_npv_test),
                                                                   np.mean (mean_npv_test) -
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)),
                                                                   np.mean (mean_npv_test) +
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)))
                                                                  )

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )



###Model multimetric categorization

In [ ]:
name='cat Age'
data[name]=None
data.loc[(data['Age, years'] >=60 ),(name)]= 1
data.loc[(data['Age, years'] < 60 ),(name)]= 0

name='cat LV ESD 1'
data[name]=None
data.loc[( (data['LV ESD, cm'] >= 3.1) & ((data['LV ESD, cm'] < 4.1)) ), (name)]= 1
data.loc[( (data['LV ESD, cm'] < 3.1) | ((data['LV ESD, cm'] >= 4.1)) ), (name)]= 0

name='cat LV ESD 2'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 5), (name)]= 1
data.loc[(data['LV ESD, cm'] < 5 ),(name)]= 0

name='cat RAD'
data[name]=None
data.loc[( (data['RAD, cm'] >= 4.2) & ((data['RAD, cm'] < 5.3)) ), (name)]= 1
data.loc[( (data['RAD, cm'] < 4.2) | ((data['RAD, cm'] >= 5.3)) ), (name)]= 0

name='cat QRS'
data[name]=None
data.loc[(data['QRS, ms\n'] >= 80),(name)]= 1
data.loc[(data['QRS, ms\n'] < 80 ),(name)]= 0

name='cat QT'
data[name]=None
data.loc[(data['QT, ms\n'] >= 390),(name)]= 1
data.loc[(data['QT, ms\n'] < 390 ),(name)]= 0

name='cat PQ'
data[name]=None
data.loc[( (data['PQ, ms\n'] >= 170) & ((data['PQ, ms\n'] < 210)) ), (name)]= 1
data.loc[( (data['PQ, ms\n'] < 170) | ((data['PQ, ms\n'] >= 210)) ), (name)]= 0

name='cat RR 1'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 700) & ((data['RR, ms\n'] < 750)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 700) | ((data['RR, ms\n'] >= 750)) ), (name)]= 0

name='cat RR 2'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 880) & ((data['RR, ms\n'] < 1100)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 880) | ((data['RR, ms\n'] >= 1100)) ), (name)]= 0

name='cat RR 3'
data[name]=None
data.loc[ (data['RR, ms\n'] >= 1100), (name)]= 1
data.loc[ (data['RR, ms\n'] < 1100), (name)]= 0

name='cat P 1'
data[name]=None
data.loc[( (data['P, ms'] >= 100) & ((data['P, ms'] < 130)) ), (name)]= 1
data.loc[( (data['P, ms'] < 100) | ((data['P, ms'] >= 130)) ), (name)]= 0

name='cat P 2'
data[name]=None
data.loc[(data['P, ms'] >= 130),(name)]= 1
data.loc[(data['P, ms'] < 130 ),(name)]= 0


features =[ 'cat Age',  'cat RAD', 'cat LV ESD 1', "cat LV ESD 2", 'cat QRS' , 'cat QT',
            'cat RR 1', 'cat RR 2', 'cat RR 3', 'cat PQ', 'cat P 1', 'cat P 2', 'Tricuspid regurgitation'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.42
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
features.remove(y_name)
n_splits = 10

########### XGBoost
lr=0.03
m_d=2
n_e=200
spw=3
child_weight=1
sub=0.7
max_step=1
gam=2
method ='exact'
boost= 'gbtree'

mean_roc_auc_test = []
mean_sen_test = []
mean_spec_test = []
mean_f1_test = []
mean_ppv_test = []
mean_npv_test = []

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []


for j in range(100):
    rm=j+42
    np.random.seed(rm)
    # We set aside 20% for final testing
    x, x_validate, y, y_validate = train_test_split(      model_data[features],model_data[y_name],
                                                          train_size=0.8,
                                                          stratify=model_data[y_name],
                                                          random_state = rm )
    x = np.array(x)
    y = utils.to_categorical(y)
    x_validate = np.array(x_validate)
    y_validate = utils.to_categorical(y_validate)

    roc_auc_test=[]
    sen_test=[]
    spec_test=[]
    matr_test=[]
    f1_test=[]
    ppv_test=[]
    npv_test=[]
    threshold_test=[]
    skf = StratifiedKFold(n_splits=n_splits )

    # training and cross-validation
    for i, (train_index, test_index) in enumerate(skf.split(x, y[:,1])):

        x_train, x_test = x[train_index,:], x[test_index,:]
        y_train, y_test = y[train_index,:], y[test_index,:]

        model = xgb.XGBClassifier(learning_rate=lr,  eval_metric = "auc",
                                      scale_pos_weight = spw,    max_depth=m_d,  n_estimators=n_e,  random_state=rm,
                                      verbosity=0 , objective= 'binary:logistic', booster= boost,
                                      tree_method =method, max_delta_step=max_step, gamma=gam,
                                      min_child_weight=child_weight,
                                      subsample=sub, colsample_bylevel=1,
                                      n_jobs=-1, use_label_encoder=False
                                     )

        np.random.seed(rm)
        model.fit(x_train, y_train[:,1])
        y_pred_log=model.predict_proba(x_test)
        new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
        new_y = np.array(new_y)
        cMatrix = confusion_matrix(y_test[:,1].astype("int"), new_y)
        sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
        specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
        sen_test.append(sensivity)
        spec_test.append(specifity)
        fpr,tpr,thresholds = roc_curve(y_test[:,1], y_pred_log[:,1] )
        roc_auc = auc(fpr, tpr)
        roc_auc_test.append(roc_auc)
        f1=f1_score(y_test[:,1], new_y)
        f1_test.append(f1)
        ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
        npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

        ppv_test.append(ppv)
        npv_test.append(npv)

    mean_roc_auc_test.append(np.mean(roc_auc_test))
    mean_sen_test.append(np.mean(sen_test))
    mean_spec_test.append(np.mean(spec_test))
    mean_f1_test.append(np.mean(f1_test))
    mean_ppv_test.append(np.mean(ppv_test))
    mean_npv_test.append(np.mean(npv_test))

    model = xgb.XGBClassifier(learning_rate=lr,  eval_metric = "auc",
                                      scale_pos_weight = spw,    max_depth=m_d,  n_estimators=n_e,  random_state=rm,
                                      verbosity=0 , objective= 'binary:logistic', booster= boost,
                                      tree_method =method, max_delta_step=max_step, gamma=gam,
                                      min_child_weight=child_weight,
                                      subsample=sub, colsample_bylevel=1,
                                      n_jobs=-1, use_label_encoder=False
                                     )

    np.random.seed(rm)
    model.fit(x, y[:,1])

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)
    cMatrix = confusion_matrix(y_validate[:,1].astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate[:,1], y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate[:,1], new_y)
    ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Mean area under the ROC curve = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_roc_auc_test),
                                                                       np.mean (mean_roc_auc_test) -
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test)) ,
                                                                       np.mean (mean_roc_auc_test) +
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test) )
                                                                             ))
print('Mean Sensitivity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen_test),
                                                                       np.mean (mean_sen_test) -
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test)) ,
                                                                       np.mean (mean_sen_test) +
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test))
                                                                               ))
print('Mean Specificity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec_test),
                                                                       np.mean (mean_spec_test) - 1.96 * np.std(mean_spec_test) /np.sqrt(len(mean_spec_test)),
                                                                       np.mean (mean_spec_test) + 1.96 * np.std(mean_spec_test)/np.sqrt(len(mean_spec_test) ))
                                                                            )
print('Mean F1-score ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1_test),
                                                                       np.mean (mean_f1_test) - 1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test)) ,
                                                                       np.mean (mean_f1_test) +
                                                                       1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test) ))
                                                                      )

print('Mean PPV ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv_test),
                                                                       np.mean (mean_ppv_test) -
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test)) ,
                                                                       np.mean (mean_ppv_test) +
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test) ))
                                                                      )

print('Mean NPV  ={:.4f} 95% [ {:.4f}, {:.4f}]'.format(np.mean (mean_npv_test),
                                                                   np.mean (mean_npv_test) -
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)),
                                                                   np.mean (mean_npv_test) +
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)))
                                                                  )

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )



###Model medians of the groups and the centroid

In [ ]:
ame='cat Age 1'
data[name]=None
data.loc[( (data['Age, years'] >= 63) & ((data['Age, years'] < 66)) ), (name)]= 1
data.loc[( (data['Age, years'] < 63) | ((data['Age, years'] >= 66)) ), (name)]= 0
name='cat Age 2'
data[name]=None
data.loc[(data['Age, years'] >=66 ),(name)]= 1
data.loc[(data['Age, years'] < 66 ),(name)]= 0

name='cat LV ESD 1'
data[name]=None
data.loc[( (data['LV ESD, cm'] >= 3.3) & ((data['LV ESD, cm'] < 3.35)) ), (name)]= 1
data.loc[( (data['LV ESD, cm'] < 3.3) | ((data['LV ESD, cm'] >= 3.35)) ), (name)]= 0
name='cat LV ESD 2'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 3.35), (name)]= 1
data.loc[(data['LV ESD, cm'] < 3.35 ),(name)]= 0


name='cat RAD 1'
data[name]=None
data.loc[( (data['RAD, cm'] >= 4.3) & ((data['RAD, cm'] < 4.5)) ), (name)]= 1
data.loc[( (data['RAD, cm'] < 4.3) | ((data['RAD, cm'] >= 4.5)) ), (name)]= 0
name='cat RAD 2'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 4.5), (name)]= 1
data.loc[(data['LV ESD, cm'] < 4.5 ),(name)]= 0


name='cat QRS'
data[name]=None
data.loc[(data['QRS, ms\n'] >= 80),(name)]= 1
data.loc[(data['QRS, ms\n'] < 80 ),(name)]= 0


name='cat QT 1'
data[name]=None
data.loc[( (data['QT, ms\n'] >= 380) & ((data['QT, ms\n'] < 400)) ), (name)]= 1
data.loc[( (data['QT, ms\n'] < 380) | ((data['QT, ms\n'] >= 400)) ), (name)]= 0
name='cat QT 2'
data[name]=None
data.loc[(data['QT, ms\n'] >= 400),(name)]= 1
data.loc[(data['QT, ms\n'] < 400 ),(name)]= 0


name='cat PQ'
data[name]=None
data.loc[(data['PQ, ms\n'] >= 160),(name)]= 1
data.loc[(data['PQ, ms\n'] < 160 ),(name)]= 0


name='cat RR 1'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 900) & ((data['RR, ms\n'] < 950)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 900) | ((data['RR, ms\n'] >= 950)) ), (name)]= 0
name='cat RR 2'
data[name]=None
data.loc[(data['RR, ms\n'] >= 950),(name)]= 1
data.loc[(data['RR, ms\n'] < 950),(name)]= 0

name='cat P'
data[name]=None
data.loc[(data['P, ms'] >= 100),(name)]= 1
data.loc[(data['P, ms'] < 100),(name)]= 0


features =[ 'cat Age 1', 'cat Age 2',
            'cat LV ESD 1', 'cat LV ESD 2', "cat RAD 1",  'cat QRS',
            'cat QT 1', 'cat QT 2', 'cat PQ', 'cat RR 1', 'cat RR 2', 'cat P', 'Tricuspid regurgitation'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.39
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
features.remove(y_name)
n_splits = 10

########### XGBoost
lr=0.03
m_d=2
n_e=200
spw=3
child_weight=1
sub=0.7
max_step=1
gam=2
method ='exact'
boost= 'gbtree'

mean_roc_auc_test = []
mean_sen_test = []
mean_spec_test = []
mean_f1_test = []
mean_ppv_test = []
mean_npv_test = []

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []


for j in range(100):
    rm=j+42
    np.random.seed(rm)
    # We set aside 20% for final testing
    x, x_validate, y, y_validate = train_test_split(      model_data[features],model_data[y_name],
                                                          train_size=0.8,
                                                          stratify=model_data[y_name],
                                                          random_state = rm )
    x = np.array(x)
    y = utils.to_categorical(y)
    x_validate = np.array(x_validate)
    y_validate = utils.to_categorical(y_validate)

    roc_auc_test=[]
    sen_test=[]
    spec_test=[]
    matr_test=[]
    f1_test=[]
    ppv_test=[]
    npv_test=[]
    threshold_test=[]
    skf = StratifiedKFold(n_splits=n_splits )

    # training and cross-validation
    for i, (train_index, test_index) in enumerate(skf.split(x, y[:,1])):

        x_train, x_test = x[train_index,:], x[test_index,:]
        y_train, y_test = y[train_index,:], y[test_index,:]

        model = xgb.XGBClassifier(learning_rate=lr,  eval_metric = "auc",
                                      scale_pos_weight = spw,    max_depth=m_d,  n_estimators=n_e,  random_state=rm,
                                      verbosity=0 , objective= 'binary:logistic', booster= boost,
                                      tree_method =method, max_delta_step=max_step, gamma=gam,
                                      min_child_weight=child_weight,
                                      subsample=sub, colsample_bylevel=1,
                                      n_jobs=-1, use_label_encoder=False
                                     )

        np.random.seed(rm)
        model.fit(x_train, y_train[:,1])
        y_pred_log=model.predict_proba(x_test)
        new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
        new_y = np.array(new_y)
        cMatrix = confusion_matrix(y_test[:,1].astype("int"), new_y)
        sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
        specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
        sen_test.append(sensivity)
        spec_test.append(specifity)
        fpr,tpr,thresholds = roc_curve(y_test[:,1], y_pred_log[:,1] )
        roc_auc = auc(fpr, tpr)
        roc_auc_test.append(roc_auc)
        f1=f1_score(y_test[:,1], new_y)
        f1_test.append(f1)
        ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
        npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

        ppv_test.append(ppv)
        npv_test.append(npv)

    mean_roc_auc_test.append(np.mean(roc_auc_test))
    mean_sen_test.append(np.mean(sen_test))
    mean_spec_test.append(np.mean(spec_test))
    mean_f1_test.append(np.mean(f1_test))
    mean_ppv_test.append(np.mean(ppv_test))
    mean_npv_test.append(np.mean(npv_test))

    model = xgb.XGBClassifier(learning_rate=lr,  eval_metric = "auc",
                                      scale_pos_weight = spw,    max_depth=m_d,  n_estimators=n_e,  random_state=rm,
                                      verbosity=0 , objective= 'binary:logistic', booster= boost,
                                      tree_method =method, max_delta_step=max_step, gamma=gam,
                                      min_child_weight=child_weight,
                                      subsample=sub, colsample_bylevel=1,
                                      n_jobs=-1, use_label_encoder=False
                                     )

    np.random.seed(rm)
    model.fit(x, y[:,1])

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)
    cMatrix = confusion_matrix(y_validate[:,1].astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate[:,1], y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate[:,1], new_y)
    ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Mean area under the ROC curve = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_roc_auc_test),
                                                                       np.mean (mean_roc_auc_test) -
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test)) ,
                                                                       np.mean (mean_roc_auc_test) +
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test) )
                                                                             ))
print('Mean Sensitivity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen_test),
                                                                       np.mean (mean_sen_test) -
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test)) ,
                                                                       np.mean (mean_sen_test) +
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test))
                                                                               ))
print('Mean Specificity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec_test),
                                                                       np.mean (mean_spec_test) - 1.96 * np.std(mean_spec_test) /np.sqrt(len(mean_spec_test)),
                                                                       np.mean (mean_spec_test) + 1.96 * np.std(mean_spec_test)/np.sqrt(len(mean_spec_test) ))
                                                                            )
print('Mean F1-score ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1_test),
                                                                       np.mean (mean_f1_test) - 1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test)) ,
                                                                       np.mean (mean_f1_test) +
                                                                       1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test) ))
                                                                      )

print('Mean PPV ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv_test),
                                                                       np.mean (mean_ppv_test) -
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test)) ,
                                                                       np.mean (mean_ppv_test) +
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test) ))
                                                                      )

print('Mean NPV  ={:.4f} 95% [ {:.4f}, {:.4f}]'.format(np.mean (mean_npv_test),
                                                                   np.mean (mean_npv_test) -
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)),
                                                                   np.mean (mean_npv_test) +
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)))
                                                                  )

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )



###Model quartiles cutoff

In [ ]:
name='cat Age 1'
data[name]=None
data.loc[( (data['Age, years'] >= 58) & ((data['Age, years'] < 64)) ), (name)]= 1
data.loc[( (data['Age, years'] < 58) | ((data['Age, years'] >= 64)) ), (name)]= 0
name='cat Age 2'
data[name]=None
data.loc[( (data['Age, years'] >= 64) & ((data['Age, years'] < 69)) ), (name)]= 1
data.loc[( (data['Age, years'] < 64) | ((data['Age, years'] >= 69)) ), (name)]= 0
name='cat Age 3'
data[name]=None
data.loc[(data['Age, years'] >=69 ),(name)]= 1
data.loc[(data['Age, years'] < 69 ),(name)]= 0


name='cat LV ESD 1'
data[name]=None
data.loc[( (data['LV ESD, cm'] >= 3.1) & ((data['LV ESD, cm'] < 3.3)) ), (name)]= 1
data.loc[( (data['LV ESD, cm'] < 3.1) | ((data['LV ESD, cm'] >= 3.3)) ), (name)]= 0
name='cat LV ESD 2'
data[name]=None
data.loc[( (data['LV ESD, cm'] >= 3.3) & ((data['LV ESD, cm'] < 3.8)) ), (name)]= 1
data.loc[( (data['LV ESD, cm'] < 3.3) | ((data['LV ESD, cm'] >= 3.8)) ), (name)]= 0
name='cat LV ESD 3'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 3.8), (name)]= 1
data.loc[(data['LV ESD, cm'] < 3.8 ),(name)]= 0


name='cat RAD 1'
data[name]=None
data.loc[( (data['RAD, cm'] >= 3.8) & ((data['RAD, cm'] < 4.3)) ), (name)]= 1
data.loc[( (data['RAD, cm'] < 3.8) | ((data['RAD, cm'] >= 4.3)) ), (name)]= 0
name='cat RAD 2'
data[name]=None
data.loc[( (data['RAD, cm'] >= 4.3) & ((data['RAD, cm'] < 4.8)) ), (name)]= 1
data.loc[( (data['RAD, cm'] < 4.3) | ((data['RAD, cm'] >= 4.8)) ), (name)]= 0
name='cat RAD 3'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 4.8), (name)]= 1
data.loc[(data['LV ESD, cm'] < 4.8 ),(name)]= 0


name='cat QRS'
data[name]=None
data.loc[( (data['QRS, ms\n'] >= 80) & ((data['QRS, ms\n'] < 100)) ), (name)]= 1
data.loc[( (data['QRS, ms\n'] < 80) | ((data['QRS, ms\n'] >= 100)) ), (name)]= 0


name='cat QT 1'
data[name]=None
data.loc[( (data['QT, ms\n'] >= 360) & ((data['QT, ms\n'] < 395)) ), (name)]= 1
data.loc[( (data['QT, ms\n'] < 360) | ((data['QT, ms\n'] >= 395)) ), (name)]= 0
name='cat QT 2'
data[name]=None
data.loc[( (data['QT, ms\n'] >= 395) & ((data['QT, ms\n'] < 420)) ), (name)]= 1
data.loc[( (data['QT, ms\n'] < 395) | ((data['QT, ms\n'] >= 420)) ), (name)]= 0
name='cat QT 3'
data[name]=None
data.loc[(data['QT, ms\n'] >= 420),(name)]= 1
data.loc[(data['QT, ms\n'] < 420 ),(name)]= 0


name='cat RR 1'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 800) & ((data['RR, ms\n'] < 920)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 800) | ((data['RR, ms\n'] >= 920)) ), (name)]= 0
name='cat RR 2'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 920) & ((data['RR, ms\n'] < 1080)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 920) | ((data['RR, ms\n'] >= 1080)) ), (name)]= 0
name='cat RR 3'
data[name]=None
data.loc[(data['RR, ms\n'] >= 1080),(name)]= 1
data.loc[(data['RR, ms\n'] < 1080),(name)]= 0


name='cat PQ 1'
data[name]=None
data.loc[( (data['PQ, ms\n'] >= 140) & ((data['PQ, ms\n'] < 160)) ), (name)]= 1
data.loc[( (data['PQ, ms\n'] < 140) | ((data['PQ, ms\n'] >= 160)) ), (name)]= 0
name='cat PQ 2'
data[name]=None
data.loc[( (data['PQ, ms\n'] >= 160) & ((data['PQ, ms\n'] < 180)) ), (name)]= 1
data.loc[( (data['PQ, ms\n'] < 160) | ((data['PQ, ms\n'] >= 180)) ), (name)]= 0
name='cat PQ 3'
data[name]=None
data.loc[(data['PQ, ms\n'] >= 180),(name)]= 1
data.loc[(data['PQ, ms\n'] < 180 ),(name)]= 0


name='cat P'
data[name]=None
data.loc[(data['P, ms'] >= 100),(name)]= 1
data.loc[(data['P, ms'] < 100),(name)]= 0


features =[ 'cat Age 1', 'cat Age 2', 'cat Age 3',
            'cat LV ESD 1', 'cat LV ESD 2', 'cat LV ESD 3', "cat RAD 1", 'cat RAD 2', 'cat QRS', 'cat QT 2', 'cat QT 3',
            'cat RR 2', 'cat RR 3', 'cat PQ 2', 'cat PQ 3', 'cat P', 'Tricuspid regurgitation'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.40
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
features.remove(y_name)
n_splits = 10

########### XGBoost
lr=0.03
m_d=2
n_e=200
spw=3
child_weight=1
sub=0.7
max_step=1
gam=2
method ='exact'
boost= 'gbtree'

mean_roc_auc_test = []
mean_sen_test = []
mean_spec_test = []
mean_f1_test = []
mean_ppv_test = []
mean_npv_test = []

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []


for j in range(100):
    rm=j+42
    np.random.seed(rm)
    # We set aside 20% for final testing
    x, x_validate, y, y_validate = train_test_split(      model_data[features],model_data[y_name],
                                                          train_size=0.8,
                                                          stratify=model_data[y_name],
                                                          random_state = rm )
    x = np.array(x)
    y = utils.to_categorical(y)
    x_validate = np.array(x_validate)
    y_validate = utils.to_categorical(y_validate)

    roc_auc_test=[]
    sen_test=[]
    spec_test=[]
    matr_test=[]
    f1_test=[]
    ppv_test=[]
    npv_test=[]
    threshold_test=[]
    skf = StratifiedKFold(n_splits=n_splits )

    # training and cross-validation
    for i, (train_index, test_index) in enumerate(skf.split(x, y[:,1])):

        x_train, x_test = x[train_index,:], x[test_index,:]
        y_train, y_test = y[train_index,:], y[test_index,:]

        model = xgb.XGBClassifier(learning_rate=lr,  eval_metric = "auc",
                                      scale_pos_weight = spw,    max_depth=m_d,  n_estimators=n_e,  random_state=rm,
                                      verbosity=0 , objective= 'binary:logistic', booster= boost,
                                      tree_method =method, max_delta_step=max_step, gamma=gam,
                                      min_child_weight=child_weight,
                                      subsample=sub, colsample_bylevel=1,
                                      n_jobs=-1, use_label_encoder=False
                                     )

        np.random.seed(rm)
        model.fit(x_train, y_train[:,1])
        y_pred_log=model.predict_proba(x_test)
        new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
        new_y = np.array(new_y)
        cMatrix = confusion_matrix(y_test[:,1].astype("int"), new_y)
        sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
        specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
        sen_test.append(sensivity)
        spec_test.append(specifity)
        fpr,tpr,thresholds = roc_curve(y_test[:,1], y_pred_log[:,1] )
        roc_auc = auc(fpr, tpr)
        roc_auc_test.append(roc_auc)
        f1=f1_score(y_test[:,1], new_y)
        f1_test.append(f1)
        ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
        npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

        ppv_test.append(ppv)
        npv_test.append(npv)

    mean_roc_auc_test.append(np.mean(roc_auc_test))
    mean_sen_test.append(np.mean(sen_test))
    mean_spec_test.append(np.mean(spec_test))
    mean_f1_test.append(np.mean(f1_test))
    mean_ppv_test.append(np.mean(ppv_test))
    mean_npv_test.append(np.mean(npv_test))

    model = xgb.XGBClassifier(learning_rate=lr,  eval_metric = "auc",
                                      scale_pos_weight = spw,    max_depth=m_d,  n_estimators=n_e,  random_state=rm,
                                      verbosity=0 , objective= 'binary:logistic', booster= boost,
                                      tree_method =method, max_delta_step=max_step, gamma=gam,
                                      min_child_weight=child_weight,
                                      subsample=sub, colsample_bylevel=1,
                                      n_jobs=-1, use_label_encoder=False
                                     )

    np.random.seed(rm)
    model.fit(x, y[:,1])

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)
    cMatrix = confusion_matrix(y_validate[:,1].astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate[:,1], y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate[:,1], new_y)
    ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Mean area under the ROC curve = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_roc_auc_test),
                                                                       np.mean (mean_roc_auc_test) -
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test)) ,
                                                                       np.mean (mean_roc_auc_test) +
                                                                              1.96 * np.std(mean_roc_auc_test)/np.sqrt(len(mean_roc_auc_test) )
                                                                             ))
print('Mean Sensitivity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen_test),
                                                                       np.mean (mean_sen_test) -
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test)) ,
                                                                       np.mean (mean_sen_test) +
                                                                                1.96 * np.std(mean_sen_test)/np.sqrt(len(mean_sen_test))
                                                                               ))
print('Mean Specificity  ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec_test),
                                                                       np.mean (mean_spec_test) - 1.96 * np.std(mean_spec_test) /np.sqrt(len(mean_spec_test)),
                                                                       np.mean (mean_spec_test) + 1.96 * np.std(mean_spec_test)/np.sqrt(len(mean_spec_test) ))
                                                                            )
print('Mean F1-score ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1_test),
                                                                       np.mean (mean_f1_test) - 1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test)) ,
                                                                       np.mean (mean_f1_test) +
                                                                       1.96 * np.std(mean_f1_test)/np.sqrt(len(mean_f1_test) ))
                                                                      )

print('Mean PPV ={:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv_test),
                                                                       np.mean (mean_ppv_test) -
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test)) ,
                                                                       np.mean (mean_ppv_test) +
                                                                  1.96 * np.std(mean_ppv_test)/np.sqrt(len(mean_ppv_test) ))
                                                                      )

print('Mean NPV  ={:.4f} 95% [ {:.4f}, {:.4f}]'.format(np.mean (mean_npv_test),
                                                                   np.mean (mean_npv_test) -
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)),
                                                                   np.mean (mean_npv_test) +
                                                                   1.96 * np.std(mean_npv_test)/np.sqrt(len(mean_npv_test)))
                                                                  )

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )



##External validation (Table 7)


###Data load

In [72]:
## Open dataset
data_valid = pd.read_excel("DataSet_Validate.xlsx")
print(data.shape)
## How many POAF and POAF free
end_point = 'Atrial fibrillation'
print(data_valid[end_point].value_counts(dropna = False))
print(data_valid[end_point].value_counts(dropna = False, normalize=True))

(1305, 74)
Atrial fibrillation
0    161
1     39
Name: count, dtype: int64
Atrial fibrillation
0    0.805
1    0.195
Name: proportion, dtype: float64


In [73]:
name='cat Age'
data[name]=None
data.loc[(data['Age, years'] >=60 ),(name)]= 1
data.loc[(data['Age, years'] < 60 ),(name)]= 0

name='cat LV ESD 1'
data[name]=None
data.loc[( (data['LV ESD, cm'] >= 3.1) & ((data['LV ESD, cm'] < 4.1)) ), (name)]= 1
data.loc[( (data['LV ESD, cm'] < 3.1) | ((data['LV ESD, cm'] >= 4.1)) ), (name)]= 0

name='cat LV ESD 2'
data[name]=None
data.loc[(data['LV ESD, cm'] >= 5), (name)]= 1
data.loc[(data['LV ESD, cm'] < 5 ),(name)]= 0

name='cat RAD'
data[name]=None
data.loc[( (data['RAD, cm'] >= 4.2) & ((data['RAD, cm'] < 5.3)) ), (name)]= 1
data.loc[( (data['RAD, cm'] < 4.2) | ((data['RAD, cm'] >= 5.3)) ), (name)]= 0

name='cat QRS'
data[name]=None
data.loc[(data['QRS, ms\n'] >= 80),(name)]= 1
data.loc[(data['QRS, ms\n'] < 80 ),(name)]= 0

name='cat QT'
data[name]=None
data.loc[(data['QT, ms\n'] >= 390),(name)]= 1
data.loc[(data['QT, ms\n'] < 390 ),(name)]= 0

name='cat PQ'
data[name]=None
data.loc[( (data['PQ, ms\n'] >= 170) & ((data['PQ, ms\n'] < 210)) ), (name)]= 1
data.loc[( (data['PQ, ms\n'] < 170) | ((data['PQ, ms\n'] >= 210)) ), (name)]= 0

name='cat RR 1'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 700) & ((data['RR, ms\n'] < 750)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 700) | ((data['RR, ms\n'] >= 750)) ), (name)]= 0

name='cat RR 2'
data[name]=None
data.loc[( (data['RR, ms\n'] >= 880) & ((data['RR, ms\n'] < 1100)) ), (name)]= 1
data.loc[( (data['RR, ms\n'] < 880) | ((data['RR, ms\n'] >= 1100)) ), (name)]= 0

name='cat RR 3'
data[name]=None
data.loc[ (data['RR, ms\n'] >= 1100), (name)]= 1
data.loc[ (data['RR, ms\n'] < 1100), (name)]= 0

name='cat P 1'
data[name]=None
data.loc[( (data['P, ms'] >= 100) & ((data['P, ms'] < 130)) ), (name)]= 1
data.loc[( (data['P, ms'] < 100) | ((data['P, ms'] >= 130)) ), (name)]= 0

name='cat P 2'
data[name]=None
data.loc[(data['P, ms'] >= 130),(name)]= 1
data.loc[(data['P, ms'] < 130 ),(name)]= 0


name='cat Age'
data_valid[name]=None
data_valid.loc[(data_valid['Age, years'] >=60 ),(name)]= 1
data_valid.loc[(data_valid['Age, years'] < 60 ),(name)]= 0

name='cat LV ESD 1'
data_valid[name]=None
data_valid.loc[( (data_valid['LV ESD, cm'] >= 3.1) & ((data_valid['LV ESD, cm'] < 4.1)) ), (name)]= 1
data_valid.loc[( (data_valid['LV ESD, cm'] < 3.1) | ((data_valid['LV ESD, cm'] >= 4.1)) ), (name)]= 0

name='cat LV ESD 2'
data_valid[name]=None
data_valid.loc[(data_valid['LV ESD, cm'] >= 5), (name)]= 1
data_valid.loc[(data_valid['LV ESD, cm'] < 5 ),(name)]= 0

name='cat RAD'
data_valid[name]=None
data_valid.loc[( (data['RAD, cm'] >= 4.2) & ((data_valid['RAD, cm'] < 5.3)) ), (name)]= 1
data_valid.loc[( (data['RAD, cm'] < 4.2) | ((data_valid['RAD, cm'] >= 5.3)) ), (name)]= 0

name='cat QRS'
data_valid[name]=None
data_valid.loc[(data_valid['QRS, ms\n'] >= 80),(name)]= 1
data_valid.loc[(data_valid['QRS, ms\n'] < 80 ),(name)]= 0

name='cat QT'
data_valid[name]=None
data_valid.loc[(data_valid['QT, ms\n'] >= 390),(name)]= 1
data_valid.loc[(data_valid['QT, ms\n'] < 390 ),(name)]= 0

name='cat PQ'
data_valid[name]=None
data_valid.loc[( (data_valid['PQ, ms\n'] >= 170) & ((data_valid['PQ, ms\n'] < 210)) ), (name)]= 1
data_valid.loc[( (data_valid['PQ, ms\n'] < 170) | ((data_valid['PQ, ms\n'] >= 210)) ), (name)]= 0

name='cat RR 1'
data_valid[name]=None
data_valid.loc[( (data_valid['RR, ms\n'] >= 700) & ((data_valid['RR, ms\n'] < 750)) ), (name)]= 1
data_valid.loc[( (data_valid['RR, ms\n'] < 700) | ((data_valid['RR, ms\n'] >= 750)) ), (name)]= 0

name='cat RR 2'
data_valid[name]=None
data_valid.loc[( (data_valid['RR, ms\n'] >= 880) & ((data_valid['RR, ms\n'] < 1100)) ), (name)]= 1
data_valid.loc[( (data_valid['RR, ms\n'] < 880) | ((data_valid['RR, ms\n'] >= 1100)) ), (name)]= 0

name='cat RR 3'
data_valid[name]=None
data_valid.loc[ (data_valid['RR, ms\n'] >= 1100), (name)]= 1
data_valid.loc[ (data_valid['RR, ms\n'] < 1100), (name)]= 0

name='cat P 1'
data_valid[name]=None
data_valid.loc[( (data_valid['P, ms'] >= 100) & ((data_valid['P, ms'] < 130)) ), (name)]= 1
data_valid.loc[( (data_valid['P, ms'] < 100) | ((data_valid['P, ms'] >= 130)) ), (name)]= 0

name='cat P 2'
data_valid[name]=None
data_valid.loc[(data_valid['P, ms'] >= 130),(name)]= 1
data_valid.loc[(data_valid['P, ms'] < 130 ),(name)]= 0

###Continuous form of predictors

####Logistic regression

In [113]:

#Data preparation
features =[ 'Age, years',  'RAD, cm' , 'Tricuspid regurgitation' ,  'QRS, ms\n' , 'QT, ms\n', 'RR, ms\n' ,'PQ, ms\n', 'P, ms' ,  'LV ESD, cm'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.1775
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
model_data_valid = data_valid[features].dropna()
features.remove(y_name)
np.random.seed(rm)

########### Logistic regression
max_iter = 10000

mean_roc_auc_test = []
mean_sen_test = []
mean_spec_test = []
mean_f1_test = []
mean_ppv_test = []
mean_npv_test = []

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []

x = model_data[features].values
y = model_data[[y_name]].values

model = LogisticRegression( max_iter=max_iter )
np.random.seed(rm)
model.fit(x, y)

for j in range(2000):
    rm=j+42
    np.random.seed(rm)

    x = np.array(x)
    y = utils.to_categorical(y)
    x, x_validate, y, y_validate = train_test_split(      model_data_valid[features].values,model_data_valid[y_name].values,
                                                          train_size=0.95,
                                                          stratify=model_data_valid[y_name],
                                                          random_state = rm )

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)


    cMatrix = confusion_matrix(y_validate.astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate, y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate, new_y)
    if cMatrix[1][1] + cMatrix[0][1] > 0:
      ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    else:
      ppv = 0
    if cMatrix[0][0] + cMatrix[1][0] > 0:
      npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )
    else:
      npv=0
    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Validated area under the ROC curve = 0.6729 95% [ 0.6636, 0.6823]
Validated Sensitivity  = 0.5597 95% [ 0.5446,  0.5749]
Validated Specificity  = 0.6006 95% [ 0.5933,  0.6080]
Validated F1-score  =  0.3481 95% [ 0.3387,  0.3576]
Validated PPV  = 0.2636 95% [ 0.2557,  0.2715]
Validated NPV  = 0.8498 95% [ 0.8446,  0.8551]


####XGBoost

In [116]:
#Data preparation
features =[ 'Age, years',  'RAD, cm' , 'Tricuspid regurgitation' ,  'QRS, ms\n' , 'QT, ms\n', 'RR, ms\n' ,'PQ, ms\n', 'P, ms' ,  'LV ESD, cm'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.415
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
model_data_valid = data_valid[features].dropna()
features.remove(y_name)
np.random.seed(rm)

########### XGBoost
lr=0.03
m_d=2
n_e=50
spw=3
child_weight=1
sub=0.5
max_step=1
gam=2
method ='exact'
boost= 'gbtree'

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []

x = model_data[features].values
y = model_data[[y_name]].values

model = xgb.XGBClassifier(learning_rate=lr,  eval_metric = "auc",
                                      scale_pos_weight = spw,    max_depth=m_d,  n_estimators=n_e,  random_state=rm,
                                      verbosity=0 , objective= 'binary:logistic', booster= boost,
                                      tree_method =method, max_delta_step=max_step, gamma=gam,
                                      min_child_weight=child_weight,
                                      subsample=sub, colsample_bylevel=1,
                                      n_jobs=-1, use_label_encoder=False
                                     )
np.random.seed(rm)
model.fit(x, y)

for j in range(2000):
    rm=j+42
    np.random.seed(rm)

    x = np.array(x)
    y = utils.to_categorical(y)
    x, x_validate, y, y_validate = train_test_split(      model_data_valid[features],model_data_valid[y_name],
                                                          train_size=0.95,
                                                          stratify=model_data_valid[y_name],
                                                          random_state = rm )

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)

    cMatrix = confusion_matrix(y_validate.astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate, y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate, new_y)
    if cMatrix[1][1] + cMatrix[0][1] > 0:
      ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    else:
      ppv = 0
    if cMatrix[0][0] + cMatrix[1][0] > 0:
      npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )
    else:
      ppv = 0

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )

Validated area under the ROC curve = 0.7870 95% [ 0.7793, 0.7946]
Validated Sensitivity  = 0.6983 95% [ 0.6842,  0.7123]
Validated Specificity  = 0.6885 95% [ 0.6813,  0.6957]
Validated F1-score  =  0.4758 95% [ 0.4662,  0.4853]
Validated PPV  = 0.3845 95% [ 0.3751,  0.3938]
Validated NPV  = 0.9075 95% [ 0.9031,  0.9118]


####Random Forest

In [123]:
#Data preparation
features =[ 'Age, years',  'RAD, cm' , 'Tricuspid regurgitation' ,  'QRS, ms\n' , 'QT, ms\n', 'RR, ms\n' ,'PQ, ms\n', 'P, ms' ,  'LV ESD, cm'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.20
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
model_data_valid = data_valid[features].dropna()
features.remove(y_name)
np.random.seed(rm)

########### Random Forest
m_d1=2 #8
n_e1=100  #110
min_leaf = 0.01 #0.01

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []

x = model_data[features].values
y = model_data[[y_name]].values

model = RandomForestClassifier(random_state=rm, n_estimators=n_e1, max_depth=m_d1
                                          ,min_samples_leaf = min_leaf
                                          )
np.random.seed(rm)
model.fit(x, y)

for j in range(2000):
    rm=j+42
    np.random.seed(rm)

    x = np.array(x)
    y = utils.to_categorical(y)
    x, x_validate, y, y_validate = train_test_split(      model_data_valid[features].values,model_data_valid[y_name].values,
                                                          train_size=0.95,
                                                          stratify=model_data_valid[y_name],
                                                          random_state = rm )

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)

    cMatrix = confusion_matrix(y_validate.astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate, y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate, new_y)
    if cMatrix[1][1] + cMatrix[0][1] > 0:
      ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    else:
      ppv = 0
    if cMatrix[0][0] + cMatrix[1][0] > 0:
      npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )
    else:
      ppv = 0

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )



/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Validated area under the ROC curve = 0.7656 95% [ 0.7575, 0.7738]
Validated Sensitivity  = 0.7650 95% [ 0.7519,  0.7781]
Validated Specificity  = 0.6442 95% [ 0.6371,  0.6514]
Validated F1-score  =  0.4842 95% [ 0.4756,  0.4927]
Validated PPV  = 0.3700 95% [ 0.3622,  0.3778]
Validated NPV  = 0.9232 95% [ 0.9188,  0.9275]


###Categorized form of predictors

####Logistic regression

In [132]:
#Data preparation
features =[ 'cat Age',  'cat RAD', 'cat LV ESD 1', "cat LV ESD 2", 'cat QRS' , 'cat QT',
            'cat RR 1', 'cat RR 2', 'cat RR 3', 'cat PQ', 'cat P 1', 'cat P 2', 'Tricuspid regurgitation'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.22
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
model_data_valid = data_valid[features].dropna()
features.remove(y_name)
np.random.seed(rm)

########### Logistic regression
max_iter = 10000

mean_roc_auc_test = []
mean_sen_test = []
mean_spec_test = []
mean_f1_test = []
mean_ppv_test = []
mean_npv_test = []

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []

x = model_data[features].values
y = model_data[[y_name]].values

model = LogisticRegression( max_iter=max_iter )
np.random.seed(rm)
model.fit(x, y)

for j in range(2000):
    rm=j+42
    np.random.seed(rm)

    x = np.array(x)
    y = utils.to_categorical(y)
    x, x_validate, y, y_validate = train_test_split(      model_data_valid[features].values,model_data_valid[y_name].values,
                                                          train_size=0.95,
                                                          stratify=model_data_valid[y_name],
                                                          random_state = rm )

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)


    cMatrix = confusion_matrix(y_validate.astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate, y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate, new_y)
    if cMatrix[1][1] + cMatrix[0][1] > 0:
      ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    else:
      ppv = 0
    npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Validated area under the ROC curve = 0.7725 95% [ 0.7643, 0.7807]
Validated Sensitivity  = 0.7093 95% [ 0.6951,  0.7234]
Validated Specificity  = 0.6929 95% [ 0.6859,  0.6999]
Validated F1-score  =  0.4849 95% [ 0.4751,  0.4948]
Validated PPV  = 0.3905 95% [ 0.3811,  0.3999]
Validated NPV  = 0.9107 95% [ 0.9064,  0.9151]


####XGBoost

In [134]:
#Data preparation
features =[ 'cat Age',  'cat RAD', 'cat LV ESD 1', "cat LV ESD 2", 'cat QRS' , 'cat QT',
            'cat RR 1', 'cat RR 2', 'cat RR 3', 'cat PQ', 'cat P 1', 'cat P 2', 'Tricuspid regurgitation'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.445
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
model_data_valid = data_valid[features].dropna()
model_data_valid = model_data_valid.astype( int )
features.remove(y_name)
np.random.seed(rm)

########### XGBoost
lr=0.03
m_d=2
n_e=100
spw=3
child_weight=1
sub=0.5
max_step=1
gam=2
method ='exact'
boost= 'gbtree'

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []

x = model_data[features].values
y = model_data[[y_name]].values

model = xgb.XGBClassifier(learning_rate=lr,  eval_metric = "auc",
                                      scale_pos_weight = spw,    max_depth=m_d,  n_estimators=n_e,  random_state=rm,
                                      verbosity=0 , objective= 'binary:logistic', booster= boost,
                                      tree_method =method, max_delta_step=max_step, gamma=gam,
                                      min_child_weight=child_weight,
                                      subsample=sub, colsample_bylevel=1,
                                      n_jobs=-1, use_label_encoder=False
                                     )
np.random.seed(rm)
model.fit(x, y)

for j in range(2000):
    rm=j+42
    np.random.seed(rm)

    x = np.array(x)
    y = utils.to_categorical(y)
    x, x_validate, y, y_validate = train_test_split(      model_data_valid[features],model_data_valid[y_name],
                                                          train_size=0.95,
                                                          stratify=model_data_valid[y_name],
                                                          random_state = rm )

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)


    cMatrix = confusion_matrix(y_validate.astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate, y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate, new_y)
    if cMatrix[1][1] + cMatrix[0][1] > 0:
      ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    else:
      ppv = 0
    if cMatrix[0][0] + cMatrix[1][0] > 0:
      npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )
    else:
      ppv = 0


    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )

Validated area under the ROC curve = 0.7747 95% [ 0.7667, 0.7828]
Validated Sensitivity  = 0.7147 95% [ 0.7007,  0.7288]
Validated Specificity  = 0.7169 95% [ 0.7102,  0.7237]
Validated F1-score  =  0.5043 95% [ 0.4941,  0.5145]
Validated PPV  = 0.4135 95% [ 0.4035,  0.4236]
Validated NPV  = 0.9153 95% [ 0.9111,  0.9194]


####Random Forest

In [137]:
#Data preparation
features =[ 'cat Age',  'cat RAD', 'cat LV ESD 1', "cat LV ESD 2", 'cat QRS' , 'cat QT',
            'cat RR 1', 'cat RR 2', 'cat RR 3', 'cat PQ', 'cat P 1', 'cat P 2', 'Tricuspid regurgitation'
          ]
y_name = "Atrial fibrillation"
rm = 100
border = 0.225
np.random.seed(rm)
features.append(y_name)
model_data = data[features].dropna()
model_data_valid = data_valid[features].dropna()
features.remove(y_name)
np.random.seed(rm)

########### Random Forest
m_d1=2 #8
n_e1=100  #110
min_leaf = 0.01 #0.01

mean_roc_auc = []
mean_sen = []
mean_spec = []
mean_f1 = []
mean_ppv = []
mean_npv = []

x = model_data[features].values
y = model_data[[y_name]].values

model = RandomForestClassifier(random_state=rm, n_estimators=n_e1, max_depth=m_d1
                                          ,min_samples_leaf = min_leaf
                                          )
np.random.seed(rm)
model.fit(x, y)

for j in range(2000):
    rm=j+42
    np.random.seed(rm)

    x = np.array(x)
    y = utils.to_categorical(y)
    x, x_validate, y, y_validate = train_test_split(      model_data_valid[features].values,model_data_valid[y_name].values,
                                                          train_size=0.95,
                                                          stratify=model_data_valid[y_name],
                                                          random_state = rm )

    y_pred_log=model.predict_proba(x_validate)
    new_y =np.where(y_pred_log[:,1] >=border, 1, 0)
    new_y = np.array(new_y)

    cMatrix = confusion_matrix(y_validate.astype("int"), new_y)
    sensivity = cMatrix[1][1]/(cMatrix[1][0] + cMatrix[1][1])
    specifity = cMatrix[0][0]/(cMatrix[0][0] + cMatrix[0][1])
    fpr,tpr,_ = roc_curve(y_validate, y_pred_log[:,1] )
    roc_auc = auc(fpr, tpr)
    f1=f1_score(y_validate, new_y)
    if cMatrix[1][1] + cMatrix[0][1] > 0:
      ppv=cMatrix[1][1]/ (cMatrix[1][1] + cMatrix[0][1])
    else:
      ppv = 0
    if cMatrix[0][0] + cMatrix[1][0] > 0:
      npv=cMatrix[0][0] / (cMatrix[0][0] + cMatrix[1][0] )
    else:
      ppv = 0

    mean_roc_auc.append(roc_auc)
    mean_sen.append(sensivity)
    mean_spec.append(specifity)
    mean_f1.append(f1)
    mean_ppv.append(ppv)
    mean_npv.append(npv)

print('Validated area under the ROC curve = {:.4f} 95% [ {:.4f}, {:.4f}]'.format(
    np.mean (mean_roc_auc),  np.mean (mean_roc_auc) -
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) ,
    np.mean (mean_roc_auc) +
    1.96 * np.std(mean_roc_auc)/np.sqrt(len(mean_roc_auc)) )
                                                                                             )

print('Validated Sensitivity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_sen),
                                                                       np.mean (mean_sen) -
                                                                                        1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen)) ,
                                                                       np.mean (mean_sen) + 1.96 * np.std(mean_sen)/np.sqrt(len(mean_sen))
                                                                                       ))
print('Validated Specificity  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_spec),
                                                                       np.mean (mean_spec) -
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec)) ,
                                                                       np.mean (mean_spec) +
                                                                                     1.96 * np.std(mean_spec)/np.sqrt(len(mean_spec))
                                                                                    ))
print('Validated F1-score  =  {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_f1),
                                                                       np.mean (mean_f1) - 1.96 * np.std(mean_f1)/np.sqrt(len(mean_f1)) ,
                                                                       np.mean (mean_f1) + 1.96 * np.std(mean_f1) /np.sqrt(len(mean_f1))
                                                                                ))
print('Validated PPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(np.mean (mean_ppv),
                                                                       np.mean (mean_ppv) - 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv)) ,
                                                                       np.mean (mean_ppv) + 1.96 * np.std(mean_ppv)/np.sqrt(len(mean_ppv))
                                                                          ))
print('Validated NPV  = {:.4f} 95% [ {:.4f},  {:.4f}]'.format(
    np.mean (mean_npv), np.mean (mean_npv) -
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv)) ,
    np.mean (mean_npv) +
    1.96 * np.std(mean_npv)/np.sqrt(len(mean_npv) ))
     )



/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Validated area under the ROC curve = 0.7572 95% [ 0.7491, 0.7653]
Validated Sensitivity  = 0.7147 95% [ 0.7007,  0.7288]
Validated Specificity  = 0.7039 95% [ 0.6971,  0.7106]
Validated F1-score  =  0.4945 95% [ 0.4846,  0.5044]
Validated PPV  = 0.4001 95% [ 0.3905,  0.4096]
Validated NPV  = 0.9138 95% [ 0.9095,  0.9180]
